`Consideraciones inciales`


In [17]:
# crear carpeta dataset y subcarpeta data
#!mkdir dataset
#!cd dataset
#!mkdir data

In [18]:
#!pip install pyspark
#!pip install dropbox

Módulos a importar

In [19]:
import os
import requests
import zipfile
import io
import csv

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import expr
from pyspark.sql.functions import col

In [21]:
# Crear una sesión de Spark
spark = SparkSession.builder.getOrCreate()

Los códigos de estado HTTP son números de tres dígitos que se utilizan para indicar el resultado de una solicitud al servidor. Algunos ejemplos comunes de códigos de estado son:

    200 OK: La solicitud ha sido exitosa.
    400 Bad Request: La solicitud no se pudo entender o contiene sintaxis incorrecta.
    401 Unauthorized: La solicitud requiere autenticación o el usuario no tiene los permisos necesarios.
    404 Not Found: El recurso solicitado no se encuentra en el servidor.
    500 Internal Server Error: El servidor encontró una condición inesperada que le impidió cumplir con la solicitud.

# Esperanza de vida al nacer, total (años)

La esperanza de vida al nacer indica la cantidad de años que viviría un recién nacido si los patrones de mortalidad vigentes al momento de su nacimiento no cambian a lo largo de la vida del infante.

https://datos.bancomundial.org/indicador/SP.DYN.LE00.IN?view=chart

In [22]:
url_00 = "https://api.worldbank.org/v2/es/indicator/SP.DYN.LE00.IN?downloadformat=csv"

In [23]:
response_url_00 = requests.get(url_00)

In [44]:
if response_url_00.status_code == 200:
# Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_00.content), "r") as zip_url_00:
# Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_00.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_00.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [45]:
# Ruta del archivo CSV

csv_url_00 = "dataset/API_SP.DYN.LE00.IN_DS2_es_csv_v2_5466462.csv"

In [89]:
data_00 = spark.read.csv("dataset/API_SP.DYN.LE00.IN_DS2_es_csv_v2_5466462.csv")

In [93]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window

# Asignar un número de fila a cada fila del DataFrame
df_with_row_number = data_00.withColumn("row_number", monotonically_increasing_id())

# Filtrar las filas donde el número de fila sea mayor que 1
df_filtered = df_with_row_number.filter(df_with_row_number.row_number > 1 )

# Eliminar la columna "row_number" si es necesario
df_filtered = df_filtered.drop("row_number")

In [94]:
header = df_filtered.first()
data_00 = df_filtered.filter(df_filtered != header).toDF(*header)

PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `condition` should be a Column or str, got bool.

In [92]:
df_filtered.show()

+--------------------+------------+--------------------+
|                 _c0|         _c1|                 _c2|
+--------------------+------------+--------------------+
|        Country Name|Country Code|      Indicator Name|
|               Aruba|         ABW|Esperanza de vida...|
|                null|         AFE|Esperanza de vida...|
|          Afganistán|         AFG|Esperanza de vida...|
|                null|         AFW|Esperanza de vida...|
|              Angola|         AGO|Esperanza de vida...|
|             Albania|         ALB|Esperanza de vida...|
|             Andorra|         AND|Esperanza de vida...|
|      El mundo árabe|         ARB|Esperanza de vida...|
|Emiratos Árabes U...|         ARE|Esperanza de vida...|
|           Argentina|         ARG|Esperanza de vida...|
|             Armenia|         ARM|Esperanza de vida...|
|     Samoa Americana|         ASM|Esperanza de vida...|
|   Antigua y Barbuda|         ATG|Esperanza de vida...|
|           Australia|         

In [88]:
# Agregar una columna de índice al DataFrame original
data_with_index = df_filtered.rdd.zipWithIndex().map(lambda x: list(x[0]) + [x[1] + 1]).toDF(data_00.columns + ["row_number"])

# Obtener la primera fila del DataFrame original como lista
header = data_with_index.first()

# Crear un nuevo DataFrame a partir de los datos restantes
data_without_header = data_with_index.filter(data_with_index.row_number > 1)

# Establecer los nombres de columna manualmente
new_data_00 = data_without_header.toDF(*[str(col) for col in header])

# Mostrar el nuevo DataFrame
new_data_00.limit(5).pandas_api()


23/06/11 10:13:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Source, Indicadores del desarrollo mundial, 
 Schema: Data Source, Indicadores del desarrollo mundial, _c2
Expected: _c2 but found: 
CSV file: file:///_disk_dev/final_project/dataset/API_SP.DYN.LE00.IN_DS2_es_csv_v2_5466462.csv
23/06/11 10:13:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Source, Indicadores del desarrollo mundial, 
 Schema: Data Source, Indicadores del desarrollo mundial, _c2
Expected: _c2 but found: 
CSV file: file:///_disk_dev/final_project/dataset/API_SP.DYN.LE00.IN_DS2_es_csv_v2_5466462.csv
23/06/11 10:13:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Data Source, Indicadores del desarrollo mundial, 
 Schema: Data Source, Indicadores del desarrollo mundial, _c2
Expected: _c2 but found: 
CSV file: file:///_disk_dev/final_project/dataset/API_SP.DYN.LE00.IN_DS2_es_csv_v2_5466462.csv


Aruba  ABW  Esperanza de vida al nacer, total (años)  1
0        None  AFE  Esperanza de vida al nacer, total (años)  2
1  Afganistán  AFG  Esperanza de vida al nacer, total (años)  3
2        None  AFW  Esperanza de vida al nacer, total (años)  4
3      Angola  AGO  Esperanza de vida al nacer, total (años)  5
4     Albania  ALB  Esperanza de vida al nacer, total (años)  6

In [83]:
# Agregar una columna de índice al DataFrame original
data_with_index = df_filtered.rdd.zipWithIndex().map(lambda x: list(x[0]) + [x[1] + 1]).toDF(data_00.columns + ["row_number"])

# Obtener la primera fila del DataFrame original como lista
header = data_with_index.first()

# Crear un nuevo DataFrame a partir de los datos restantes
data_without_header = data_with_index.filter(data_with_index.row_number > 1)

# Establecer los nombres de columna manualmente
new_data_00 = data_without_header.toDF(*[str(col) for col in header])

# Eliminar la última columna del nuevo DataFrame
new_data_00 = new_data_00.drop("1")

# Mostrar el nuevo DataFrame
new_data_00.columns

['Country Name', 'Country Code', 'Indicator Name']

In [81]:
data_00 = new_data_00.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, \
'2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, life_expectancy)")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `1960` cannot be resolved. Did you mean one of the following? [`Country Code`, `Country Name`, `Indicator Name`].; line 1 pos 18;
'Project [Country Name#1388, Country Code#1389, Indicator Name#1390, 'stack(63, 1960, '1960, 1961, '1961, 1962, '1962, 1963, '1963, 1964, '1964, 1965, '1965, 1966, '1966, 1967, '1967, 1968, '1968, 1969, '1969, 1970, '1970, 1971, '1971, 1972, '1972, 1973, '1973, 1974, '1974, 1975, '1975, 1976, '1976, 1977, '1977, 1978, '1978, 1979, '1979, 1980, '1980, 1981, '1981, 1982, '1982, 1983, '1983, 1984, '1984, 1985, '1985, 1986, '1986, 1987, '1987, 1988, '1988, 1989, '1989, 1990, '1990, 1991, '1991, 1992, '1992, 1993, '1993, 1994, '1994, 1995, '1995, 1996, '1996, 1997, '1997, 1998, '1998, 1999, '1999, 2000, '2000, 2001, '2001, 2002, '2002, 2003, '2003, 2004, '2004, 2005, '2005, 2006, '2006, 2007, '2007, 2008, '2008, 2009, '2009, 2010, '2010, 2011, '2011, 2012, '2012, 2013,

In [79]:
data_00 = new_data_00.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, \
'2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, life_expectancy)")


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `1960` cannot be resolved. Did you mean one of the following? [`Country Code`, `Country Name`, `Indicator Name`].; line 1 pos 18;
'Project [Country Name#1388, Country Code#1389, Indicator Name#1390, 'stack(63, 1960, '1960, 1961, '1961, 1962, '1962, 1963, '1963, 1964, '1964, 1965, '1965, 1966, '1966, 1967, '1967, 1968, '1968, 1969, '1969, 1970, '1970, 1971, '1971, 1972, '1972, 1973, '1973, 1974, '1974, 1975, '1975, 1976, '1976, 1977, '1977, 1978, '1978, 1979, '1979, 1980, '1980, 1981, '1981, 1982, '1982, 1983, '1983, 1984, '1984, 1985, '1985, 1986, '1986, 1987, '1987, 1988, '1988, 1989, '1989, 1990, '1990, 1991, '1991, 1992, '1992, 1993, '1993, 1994, '1994, 1995, '1995, 1996, '1996, 1997, '1997, 1998, '1998, 1999, '1999, 2000, '2000, 2001, '2001, 2002, '2002, 2003, '2003, 2004, '2004, 2005, '2005, 2006, '2006, 2007, '2007, 2008, '2008, 2009, '2009, 2010, '2010, 2011, '2011, 2012, '2012, 2013,

In [67]:
df_filtered.limit(3).pandas_api()

_c0           _c1                                       _c2
0  Country Name  Country Code                            Indicator Name
1         Aruba           ABW  Esperanza de vida al nacer, total (años)
2          None           AFE  Esperanza de vida al nacer, total (años)

In [ ]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Asignar un número de fila a cada fila del DataFrame
window_spec = Window.orderBy("row_number")
df_with_row_number = df.withColumn("row_number", row_number().over(window_spec))

# Filtrar las filas donde el número de fila sea mayor que 3
df_filtered = df_with_row_number.filter(df_with_row_number.row_number > 3)

# Eliminar la columna "row_number" si es necesario
df_filtered = df_filtered.drop("row_number")

In [29]:
rdd_csv_url_00 = spark.read.text(csv_url_00).rdd.filter(lambda x: x[0].isdigit() and int(x[0]) >= 4)

In [31]:
#data_00 = spark.createDataFrame(rdd_csv_url_00)

print(rdd_csv_url_00.collect())

[]


In [26]:
# Crear una lista para almacenar las filas del archivo CSV
rows_url_00 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_00, "r") as file_csv_url_00:
    csv_reader_url_00 = csv.reader(file_csv_url_00)
    rows_url_00 = list(csv_reader_url_00)

# Eliminar las primeras 4 filas
rows_url_00 = rows_url_00[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_00.csv", "w", newline="") as file_csv_url_00:
    csv_writer_url_00 = csv.writer(file_csv_url_00)
    csv_writer_url_00.writerows(rows_url_00)

In [10]:
data_00 = spark.read.csv("dataset/data/data_00.csv", header=True)
data_00 = data_00.drop("_c66","_c67")

In [11]:
data_00.limit(3).pandas_api()

/usr/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
23/06/11 08:07:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Country Name Country Code                            Indicator Name  Indicator Code              1960             1961              1962              1963              1964              1965             1966              1967              1968              1969              1970              1971              1972              1973              1974              1975              1976              1977              1978             1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993             1994              1995              1996              1997              1998              1999              2000             2001             2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012             2013             2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN            64.152           64.537            64.752            65.132            65.294            65.502           66.063            66.439            66.757            67.168            67.583            67.975            68.577            69.092            69.503            69.762            70.035            70.264            70.494           70.778            71.066            71.722            71.959            72.105            72.251            72.388            72.462            72.789            73.047            73.023            73.076              73.1            73.179            73.225           73.272            73.349            73.448            73.452            73.491            73.561            73.569           73.647           73.726            73.752            73.576            73.811            74.026             74.21            74.147             74.56            75.404            75.465            75.531           75.636           75.601            75.683            75.617            75.903            76.072            76.248            75.723            74.626  None
1         None          AFE  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN  44.0855518454965  44.386696928982  44.7521818697994  44.9131593023895  45.4790430350925  45.4983377851245  45.249104629175  45.9249050028634  46.2230965735857  46.4323030920124  46.7184845197579  47.1929422628056  46.8973892484582  47.6923174333564  47.5980637543936  47.7598869102852  48.3495880515137  48.6359051941044  48.7636045995518  49.261335737194  49.6365376633939  50.0570732913929  50.2968491379812  48.7033305923076  48.6526645552392  49.0116307202704  49.6397190079537  50.0758875408274  49.3597265265243  50.6840997852884  50.6077275337211  50.3904596093263  49.9621136475362  50.2736279712378  50.882581820857  51.0019252899341  50.8106889705148  50.9742313589474  50.3259127442221  51.2378519083819  51.9644813247726  52.189648214333  52.540794004192  53.0220345898659  53.5454571098034  54.2196508058582  55.1505461100496  55.9338017149142  56.6804188973962  57.6208515442432  58.4111497501441  59.2932713889337  60.0507802468673  60.709863140516  61.337911447657  61.8564524336049  62.4440448894169  62.9223846035263  63.3658579512383  63.7556736140197  63.3138559861117  62.4545852699516  None
2   Afganistán          AFG  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN            32.535           33.068            33.547            34.016            34.494            34.953           35.453            35.924            36.418             36.91            37.418            37.923            38.444            39.003             39.55              40.1            40.645            4

In [12]:
print(data_00.columns)

['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [13]:
data_00 = data_00.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2000', `2000`, '2001', `2001`, '2002', `2002`, '2003', `2003`, '2004', `2004`, \
'2005', `2005`, '2006', `2006`, '2007', `2007`, '2008', `2008`, '2009', `2009`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, life_expatancy)")

In [14]:
data_00.limit(3).pandas_api()

Country Name Country Code                            Indicator Name  Indicator Code  Year life_expatancy
0        Aruba          ABW  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN  1960         64.152
1        Aruba          ABW  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN  1961         64.537
2        Aruba          ABW  Esperanza de vida al nacer, total (años)  SP.DYN.LE00.IN  1962         64.752

In [15]:
data_00 = data_00.drop("Indicator Name","Indicator Code")

# Esperanza de vida al nacer, varones (años)

La esperanza de vida al nacer indica la cantidad de años que viviría un recién nacido si los patrones de mortalidad vigentes al momento de su nacimiento no cambian a lo largo de la vida del infante.

https://datos.bancomundial.org/indicator/SP.DYN.LE00.MA.IN?view=chart


In [16]:
url_01 ="https://api.worldbank.org/v2/es/indicator/SP.DYN.LE00.MA.IN?downloadformat=csv"

In [17]:
response_url_01 = requests.get(url_01)

In [18]:
if response_url_01.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_01.content), "r") as zip_url_01:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_01.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_01.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [19]:
# Ruta del archivo CSV
csv_url_01 = "dataset/API_SP.DYN.LE00.MA.IN_DS2_es_csv_v2_5501246.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_01 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_01, "r") as file_csv_url_01:
    csv_reader_url_01 = csv.reader(file_csv_url_01)
    rows_url_01 = list(csv_reader_url_01)

# Eliminar las primeras 4 filas
rows_url_01 = rows_url_01[3:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_01.csv", "w", newline="") as file_csv_url_01:
    csv_writer_url_01 = csv.writer(file_csv_url_01)
    csv_writer_url_01.writerows(rows_url_01)

In [20]:
data_01 = spark.read.csv("dataset/data/data_01.csv", header=True)

In [21]:
data_01 = data_01.drop("_c66","_c67")

In [22]:
data_01.limit(3).pandas_api()


Country Name Country Code                              Indicator Name     Indicator Code              1960              1961              1962              1963              1964              1965              1966              1967              1968              1969              1970             1971              1972              1973              1974              1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990             1991              1992              1993              1994            1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN            60.582             60.88            61.023            61.372            61.522            61.686            62.329            62.702            62.966            63.335            63.715           64.073            64.516            64.868            65.259            65.428            65.807             66.14            66.491            66.908            67.372            68.178            68.456            68.587            68.751            68.861            68.954            69.581            70.082            70.039            70.096           70.142            70.177            70.223             70.27          70.311            70.447            70.491             70.54            70.591            70.597            70.724            70.817             70.91             70.63            70.965            71.384            71.797            71.256            71.644            72.126            72.223            72.346            72.361            72.448            72.569            72.582            72.838            73.096            73.286            72.828            71.798  None
1         None          AFE  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN  42.5431627109112  42.7429203642934  43.1134272779315  43.1577660426942  43.7050555142663  43.6826894478482  43.4901619048218  44.1149282976705  44.4032620497399  44.5792318064182  44.8664574804166  45.289742041229  44.9977009823533  45.8845481825658  45.8106928597236  46.0004021974988  46.4799182704524  46.7053081652754  46.7388753685675  47.2851743112482  47.6924367667017  48.0392527793917  48.2469716773968  46.6509666694814  46.4401788685541  46.7983927211196  47.3735998846512  47.6959634809497  46.9193669772453  48.3707975811769  48.2336956594134  48.103799345053  47.6512676870803  47.9092598236726  48.7719685025176  48.83997573395  48.6764095511719  48.9097550234422  48.0877684845754  49.1074005198067  50.2368403625047  50.5732792132029  50.9383927817432  51.5061246422434  52.0291796349508  52.6792907826266  53.5822670689842  54.2943348209578  54.9409629503684  55.7571338969576  56.4796472634335  57.2591451349442  57.9297138124994  58.4716979727039  59.0281376974846  59.5174587951231  60.0212437222169  60.5358346413935  60.9051812348701  61.2907170080319  60.8174801969809  59.8945050965862  None
2   Afganistán          AFG  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN             31.87            32.409            32.883            33.346            33.828            34.289            34.768            35.225            35.712            36.206            36.692           37.178            37.681            38.241            38.792             39.

In [23]:
data_01 = data_01.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, life_expatancy_male)")

In [24]:
data_01.limit(3).pandas_api()

Country Name Country Code                              Indicator Name     Indicator Code  Year life_expatancy_male
0        Aruba          ABW  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN  1960              60.582
1        Aruba          ABW  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN  1961               60.88
2        Aruba          ABW  Esperanza de vida al nacer, varones (años)  SP.DYN.LE00.MA.IN  1962              61.023

In [25]:
data_01 = data_01.drop("Country Name","Indicator Name","Indicator Code")

# Esperanza de vida al nacer, mujeres (años)
Algunos metadatos están solo en inglés

La esperanza de vida al nacer indica la cantidad de años que viviría un recién nacido si los patrones de mortalidad vigentes al momento de su nacimiento no cambian a lo largo de la vida del infante.

https://datos.bancomundial.org/indicador/SP.DYN.LE00.FE.IN?view=chart

In [26]:
url_02 ="https://api.worldbank.org/v2/es/indicator/SP.DYN.LE00.FE.IN?downloadformat=csv"

In [27]:
response_url_02 = requests.get(url_02)

In [28]:
if response_url_02.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_02.content), "r") as zip_url_02:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_02.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_02.status_code)
    

El archivo ZIP se ha descargado y descomprimido correctamente.


In [29]:
# Ruta del archivo CSV
csv_url_02 = "dataset/API_SP.DYN.LE00.FE.IN_DS2_es_csv_v2_5513518.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_02 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_02, "r") as file_csv_url_02:
    csv_reader_url_02 = csv.reader(file_csv_url_02)
    rows_url_02 = list(csv_reader_url_02)

# Eliminar las primeras 4 filas
rows_url_02 = rows_url_02[3:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_02.csv", "w", newline="") as file_csv_url_02:
    csv_writer_url_02 = csv.writer(file_csv_url_02)
    csv_writer_url_02.writerows(rows_url_02)

In [30]:
data_02 = spark.read.csv("dataset/data/data_02.csv", header=True)

In [31]:
data_02 = data_02.drop("_c67")

In [32]:
data_02.limit(3).pandas_api()


Country Name Country Code                              Indicator Name     Indicator Code              1960              1961              1962             1963              1964              1965              1966              1967              1968              1969              1970              1971              1972              1973              1974             1975              1976              1977              1978              1979              1980              1981             1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993              1994              1995              1996              1997             1998              1999              2000              2001             2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019             2020            2021  2022
0        Aruba          ABW  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN            67.783            68.269            68.579           68.992             69.18            69.449            69.901            70.281            70.673            71.135            71.585            72.006            72.785            73.487            73.909           74.284            74.417            74.525            74.615            74.746             74.83            75.298           75.504            75.707            75.855            76.052            76.139            76.117            76.113            76.147            76.196            76.147            76.181            76.218            76.253             76.36            76.403            76.355           76.378            76.466            76.477            76.495           76.555            76.503             76.45             76.56            76.542            76.478            76.944            77.371            78.612             78.62            78.603            78.803            78.614            78.644            78.486            78.794            78.854             79.01           78.445          77.315  None
1         None          AFE  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN  45.7073806114601  46.1187374476268  46.4793254952251  46.772370142972  47.3501615657085  47.4204739661519  47.1182913268798  47.8452250201486  48.1519717962566  48.3988985188352  48.6813269726716  49.2065120887183  48.9153642228006  49.5970021157428  49.4879689150577  49.619009310466  50.3314263344778  50.6851958046723  50.9293719093031  51.3649720386453  51.7179843504051  52.2219537716552  52.493403158806  50.8975139636199  51.0159667552218  51.3479765137673  52.0056410096266  52.5973633731797  51.9874975930103  53.0807189796995  53.0658395514734  52.7460551271059  52.3678484485409  52.7353252238145  53.0321322371138  53.1898367759865  52.9603591651235  53.0385817343631  52.620748145347  53.3961139305132  53.6622075489618  53.7726263118707  54.104872027507  54.4953652961009  55.0200370909508  55.7176973630047  56.6712952845485  57.5312565460398  58.3793864857342  59.4509527719329  60.3096746217232  61.3002881377364  62.1479837948472  62.9358932228363  63.6411707885006  64.1911189450575  64.8677850678876  65.3040042282935  65.8281020897847  66.2197216216821  65.836460467651  65.07846739695  None
2   Afganistán          AFG  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN            33.285            33.813            34.297           34.773            35.246            35.702            36.226            36.711            37.214            37.701            38.233             38.76              39.3            39.857            40.397           40.938           

In [33]:
data_02 = data_02.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, life_expatancy_female)")

In [34]:
data_02.limit(3).pandas_api()


Country Name Country Code                              Indicator Name     Indicator Code  Year life_expatancy_female
0        Aruba          ABW  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN  1960                67.783
1        Aruba          ABW  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN  1961                68.269
2        Aruba          ABW  Esperanza de vida al nacer, mujeres (años)  SP.DYN.LE00.FE.IN  1962                68.579

In [35]:
data_02 = data_02.drop("Country Name","Indicator Name","Indicator Code")

# Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)
Algunos metadatos están solo en inglés

La tasa de mortalidad neonatal es el número de recién nacidos que mueren antes de alcanzar los 28 días de edad, por cada 1.000 nacidos vivos en un año determinado.

https://datos.bancomundial.org/indicador/SH.DYN.NMRT?view=chart



In [36]:
url_03 = "https://api.worldbank.org/v2/es/indicator/SH.DYN.NMRT?downloadformat=csv"

In [37]:
response_url_03 = requests.get(url_03)

23/06/11 08:07:17 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [38]:
if response_url_03.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_03.content), "r") as zip_url_03:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_03.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_03.status_code)
    

El archivo ZIP se ha descargado y descomprimido correctamente.


In [39]:
# Ruta del archivo CSV
csv_url_03 = "dataset/API_SH.DYN.NMRT_DS2_es_csv_v2_5462559.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_03 = []
# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_03, "r") as file_csv_url_03:
    csv_reader_url_03 = csv.reader(file_csv_url_03)
    rows_url_03 = list(csv_reader_url_03)
    
# Eliminar las primeras 4 filas
rows_url_03 = rows_url_03[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_03.csv", "w", newline="") as file_csv_url_03:
    csv_writer_url_03 = csv.writer(file_csv_url_03)
    csv_writer_url_03.writerows(rows_url_03)

In [40]:
data_03 = spark.read.csv("dataset/data/data_03.csv", header=True)

In [41]:
data_03 = data_03.drop("_c66","_c67")

In [42]:
data_03.limit(3).pandas_api()

Country Name Country Code                                                Indicator Name Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989              1990              1991              1992              1993              1994              1995              1996              1997             1998              1999              2000             2001              2002              2003             2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016             2017              2018              2019              2020              2021  2022
0        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None              None              None              None              None              None             None              None              None             None              None              None             None              None              None              None              None              None              None              None              None              None              None              None              None             None              None              None              None              None  None
1         None          AFE   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  42.8297549630242  42.4113690560679  42.0003745337893  41.6017221913987  41.2516316717628  40.7085720917521  40.2880835884816  39.8967440749264  39.291052253688  38.6330999868164  37.9601637494138  37.160568482565  36.3835979901586  35.5361863680893  34.656877548392  33.8365654095307  33.0376461174565  32.2716635496204  31.4812178052392  30.8197722535952  30.1543631971504  29.4644187838301  28.7862930365553  28.1307423875659  27.5083932781221  26.9392074730253  26.4032313662393  25.815463081475  25.2704152491122  24.7590627470514  24.3172841944318  23.8201577306965  None
2   Afganistán          AFG   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None    79  77.4  75.9              74.4              72.9              71.3              69.8              68.4                67              65.7              64.5             63.3              62.2                61             59.7              58.5              57.2             55.9              54.5              53.2              51.8              50.4              48.9              47.4                46              44.5              43.1              41.8              40.4              39.2               38                37                36                35              34.1  None

In [43]:
data_03 = data_03.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_mort_neonatal)")

In [44]:
data_03.limit(3).pandas_api()


Country Name Country Code                                                Indicator Name Indicator Code  Year ratio_mort_neonatal
0        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1960                None
1        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1961                None
2        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1962                None

In [45]:
data_03 = data_03.drop("Country Name","Indicator Name","Indicator Code")

# Tasa de mortalidad, bebés (por cada 1.000 nacidos vivos)


La tasa de mortalidad infantil es la cantidad de infantes que mueren antes de llegar al año de vida, por cada 1000 nacidos vivos, en un año determinado.

https://datos.bancomundial.org/indicador/SP.DYN.IMRT.IN?view=chart


In [46]:
url_04 = "https://api.worldbank.org/v2/es/indicator/SP.DYN.IMRT.IN?downloadformat=csv"

In [47]:
response_url_04 = requests.get(url_04)

In [48]:
if response_url_04.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_04.content), "r") as zip_url_04:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_04.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_03.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [49]:
# Ruta del archivo CSV
csv_url_04 = "dataset/API_SP.DYN.IMRT.IN_DS2_es_csv_v2_5491556.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_04 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_04, "r") as file_csv_url_04:
    csv_reader_url_04 = csv.reader(file_csv_url_04)
    rows_url_04 = list(csv_reader_url_04)

# Eliminar las primeras 4 filas
rows_url_04 = rows_url_04[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_04.csv", "w", newline="") as file_csv_url_03:
    csv_writer_url_03 = csv.writer(file_csv_url_03)
    csv_writer_url_03.writerows(rows_url_03)

In [50]:
data_04 = spark.read.csv("dataset/data/data_04.csv", header=True)

In [51]:
data_04 = data_04.drop("_c67")

In [52]:
data_04.limit(3).pandas_api()


Country Name Country Code                                                Indicator Name Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989              1990              1991              1992              1993              1994              1995              1996              1997             1998              1999              2000             2001              2002              2003             2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016             2017              2018              2019              2020              2021  2022
0        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None              None              None              None              None              None             None              None              None             None              None              None             None              None              None              None              None              None              None              None              None              None              None              None              None             None              None              None              None              None  None
1         None          AFE   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  42.8297549630242  42.4113690560679  42.0003745337893  41.6017221913987  41.2516316717628  40.7085720917521  40.2880835884816  39.8967440749264  39.291052253688  38.6330999868164  37.9601637494138  37.160568482565  36.3835979901586  35.5361863680893  34.656877548392  33.8365654095307  33.0376461174565  32.2716635496204  31.4812178052392  30.8197722535952  30.1543631971504  29.4644187838301  28.7862930365553  28.1307423875659  27.5083932781221  26.9392074730253  26.4032313662393  25.815463081475  25.2704152491122  24.7590627470514  24.3172841944318  23.8201577306965  None
2   Afganistán          AFG   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None    79  77.4  75.9              74.4              72.9              71.3              69.8              68.4                67              65.7              64.5             63.3              62.2                61             59.7              58.5              57.2             55.9              54.5              53.2              51.8              50.4              48.9              47.4                46              44.5              43.1              41.8              40.4              39.2               38                37                36                35              34.1  None

In [53]:
data_04 = data_04.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_mort_one_year)")

In [54]:
data_04.limit(3).pandas_api()

Country Name Country Code                                                Indicator Name Indicator Code  Year ratio_mort_one_year
0        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1960                None
1        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1961                None
2        Aruba          ABW   Tasa de mortalidad, neonatal (por cada 1.000 nacidos vivos)    SH.DYN.NMRT  1962                None

In [55]:
data_04 = data_04.drop("Country Name","Indicator Name","Indicator Code")

# Tasa de mortalidad, menores de 5 años (por cada 1.000)
Algunos metadatos están solo en inglés

Menores de cinco años la tasa de mortalidad es la probabilidad por cada 1.000 que un recién nacido muera antes de cumplir cinco años de edad, si está sujeto a las tasas de mortalidad específicas por edad del año especificado.

https://datos.bancomundial.org/indicador/SH.DYN.MORT?view=chart

In [56]:
url_05 = "https://api.worldbank.org/v2/es/indicator/SH.DYN.MORT?downloadformat=csv"

In [57]:
response_url_05 = requests.get(url_05)

In [58]:
if response_url_05.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_05.content), "r") as zip_url_05:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_05.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_05.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [59]:
# Ruta del archivo CSV
csv_url_05 = "dataset/API_SH.DYN.MORT_DS2_es_csv_v2_5455726.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_05 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_05, "r") as file_csv_url_05:
    csv_reader_url_05 = csv.reader(file_csv_url_05)
    rows_url_05 = list(csv_reader_url_05)

# Eliminar las primeras 4 filas
rows_url_05 = rows_url_05[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_05.csv", "w", newline="") as file_csv_url_05:
    csv_writer_url_05 = csv.writer(file_csv_url_05)
    csv_writer_url_05.writerows(rows_url_05)

In [60]:
data_05 = spark.read.csv("dataset/data/data_05.csv", header=True)

In [61]:
data_05 = data_05.drop("_c66","_c67")

In [62]:
data_05.limit(3).pandas_api()


Country Name Country Code                                          Indicator Name Indicator Code   1960   1961   1962   1963   1964   1965   1966   1967   1968   1969   1970   1971   1972   1973   1974  1975   1976   1977   1978   1979   1980   1981   1982   1983   1984   1985   1986   1987   1988   1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999             2000              2001              2002             2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT   None   None   None   None   None   None   None   None   None   None   None   None   None   None   None  None   None   None   None   None   None   None   None   None   None   None   None   None   None   None              None              None              None              None              None              None              None              None              None              None             None              None              None             None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None  None
1         None          AFE  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT   None   None   None   None   None   None   None   None   None   None   None   None   None   None   None  None   None   None   None   None   None   None   None   None   None   None   None   None   None   None  165.832561519295  164.161054739235  162.695216656375  161.379202446574  161.983963117422  156.580561402565  153.998650706914  151.260901621137  147.410842670347  142.992355070727  138.18023473864  132.744121726178  127.135609785381  121.43551493647  115.828866802817  110.471932277019  105.457181745716  100.411263900067  95.3811815990238  90.7111291854921  86.4163803141884  82.4915063909405  78.7877273029938  75.4426897327386  72.4923836880967  69.8691858575132  67.3639290496706  65.0910528384151  62.8571893618424  60.8559168091663  59.0873088673173  57.2847270993243  None
2   Afganistán          AFG  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT  357.3  351.7  345.8  340.2  334.8  329.4  323.9  318.4  312.8  307.2  301.9  296.4  290.7  285.2  279.6   274  268.3  262.4  256.4  250.3  243.8  237.3  230.8  224.2  217.7  211.1  204.6  197.9  191.4  184.9             178.5             172.2             166.3             160.8             155.5             150.6               146             141.6             137.3             133.3            129.3             125.3             121.2              117             112.8             108.6             104.3               100              95.9              91.8              87.8                84              80.3              76.8              73.4              70.2              67.2              64.6              62.2              59.9              57.8              55.7  None

In [63]:
data_05 = data_05.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_mort_five_year)")

In [64]:
data_05.limit(3).pandas_api()


Country Name Country Code                                          Indicator Name Indicator Code  Year ratio_mort_five_year
0        Aruba          ABW  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT  1960                 None
1        Aruba          ABW  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT  1961                 None
2        Aruba          ABW  Tasa de mortalidad, menores de 5 años (por cada 1.000)    SH.DYN.MORT  1962                 None

In [65]:
data_05 = data_05.drop("Country Name","Indicator Name","Indicator Code")

# Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)
Algunos metadatos están solo en inglés

La tasa de mortalidad materna es la cantidad de mujeres que mueren durante el embarazo y el parto por cada 100.000 nacidos vivos. Los datos se estiman según un modelo de regresión que usa información sobre fertilidad, asistentes para el parto y prevalencia del VIH.

https://datos.bancomundial.org/indicator/SH.STA.MMRT?view=chart

#url_06 = "https://api.worldbank.org/v2/es/indicator/SP.DYN.IMRT.FE.IN?downloadformat=csv"

In [66]:
url_06 = "https://api.worldbank.org/v2/es/indicator/SH.STA.MMRT?downloadformat=csv"

In [67]:
response_url_06 = requests.get(url_06)

In [68]:
if response_url_06.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_06.content), "r") as zip_url_06:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_06.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_06.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [69]:
# Ruta del archivo CSV

csv_url_06 = "dataset/API_SH.STA.MMRT_DS2_es_csv_v2_5465523.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_06 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_06, "r") as file_csv_url_06:
    csv_reader_url_06 = csv.reader(file_csv_url_06)
    rows_url_06 = list(csv_reader_url_06)

# Eliminar las primeras 4 filas
rows_url_06 = rows_url_06[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_06.csv", "w", newline="") as file_csv_url_06:
    csv_writer_url_06 = csv.writer(file_csv_url_06)
    csv_writer_url_06.writerows(rows_url_06)

In [70]:
data_06 = spark.read.csv("dataset/data/data_06.csv", header=True)

In [71]:
data_06 = data_06.drop("_c66","_c67")

In [72]:
data_06.limit(3).pandas_api()


Country Name Country Code                                                                         Indicator Name Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999  2000  2001  2002  2003  2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  2022
0        Aruba          ABW  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
1         None          AFE  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None   697   684   670   640   617   598   572   560   551   529   507   485   458   442   425   410   392   381   364   354   347  None  None
2   Afganistán          AFG  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  1346  1273  1277  1196  1115  1103  1044  1023   962   913   899   884   833   821   785   776   750   682   663   644   620  None  None

In [73]:
data_06 = data_06.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_maternal_mort)")

In [74]:
data_06.limit(3).pandas_api()


Country Name Country Code                                                                         Indicator Name Indicator Code  Year ratio_maternal_mort
0        Aruba          ABW  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  1960                None
1        Aruba          ABW  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  1961                None
2        Aruba          ABW  Tasa de mortalidad materna (estimado mediante modelo, por cada 100.000 nacidos vivos)    SH.STA.MMRT  1962                None

In [75]:
data_06 = data_06.drop("Country Name","Indicator Name","Indicator Code")

# Tasa de mortalidad en un año (por cada 1.000 personas)

La tasa bruta de mortalidad es la cantidad de muertes en un año, por cada 1000 habitantes, estimada a mitad de año. Si se resta la tasa bruta de mortalidad a la tasa bruta de natalidad ofrece la tasa de incremento natural, que es igual a la tasa de crecimiento de la población en ausencia de migración. 

https://datos.bancomundial.org/indicador/SP.DYN.CDRT.IN?view=chart


In [76]:
url_07 = "https://api.worldbank.org/v2/es/indicator/SP.DYN.CDRT.IN?downloadformat=csv"

In [77]:
response_url_07 = requests.get(url_07)

In [78]:
if response_url_07.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_07.content), "r") as zip_url_07:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_07.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_07.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [79]:
# Ruta del archivo CSV

csv_url_07 = "dataset/API_SP.DYN.CDRT.IN_DS2_es_csv_v2_5464535.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_07 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_07, "r") as file_csv_url_07:
    csv_reader_url_07 = csv.reader(file_csv_url_07)
    rows_url_07 = list(csv_reader_url_07)

# Eliminar las primeras 4 filas
rows_url_07 = rows_url_07[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_07.csv", "w", newline="") as file_csv_url_07:
    csv_writer_url_07 = csv.writer(file_csv_url_07)
    csv_writer_url_07.writerows(rows_url_07)

In [80]:
data_07 = spark.read.csv("dataset/data/data_07.csv", header=True)

In [81]:
data_07 = data_07.drop("_c66","_c67")

In [82]:
data_07.limit(3).pandas_api()


Country Name Country Code                                          Indicator Name  Indicator Code              1960              1961              1962              1963              1964              1965              1966              1967              1968              1969              1970             1971              1972              1973             1974              1975              1976              1977              1978              1979              1980             1981              1982              1983              1984              1985              1986              1987             1988              1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005             2006              2007              2008             2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019            2020              2021  2022
0        Aruba          ABW  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN             6.594             6.456             6.394             6.275              6.25             6.216             6.047             5.967             5.957             5.916              5.88            5.861             5.754             5.677            5.657             5.699             5.741             5.838             5.934                 6              6.06            5.935             6.118             6.253             6.366             6.498              6.69              6.78            6.902             7.129               7.1             6.869             6.418             6.411             6.388             6.366             6.338             6.359             6.339             6.313             6.335             6.348              6.42             6.549             6.808             6.839            6.887             6.958             7.179            7.094             6.745             6.945             7.187             7.403             7.729             7.968             8.336             8.414             8.597             8.778            9.69            11.235  None
1         None          AFE  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN  20.9181075540334  20.6814037084123  20.3931182132739  20.2546340226112  19.8084915885047  19.7955586733842  20.0475420648391  19.4645380816404  19.2369192473959  19.0942882435246  18.8765433133638  18.474162520435  18.8197995486089  18.1259370849401  18.237209408909  18.0930053686215  17.5592278381769  17.3564034992962  17.2442005722328  16.9003662966933  16.6805167351597  16.443989250957  16.2441479956268  17.6213835986256  17.5989568342311  17.2709323329327  16.7716874800497  16.3449467825335  17.224145717891  15.6406540782482  15.6053387597827  15.8224838028085  16.0772796978922  15.6194055296085  16.2807659084593  14.8200950507174  14.8321153104661  14.6342154835785  15.1479185859315  14.2454433701731  13.7756980933728  13.5923980889675  13.3095890478815  12.9812714657523  12.6284413203686  12.2334892737562  11.731215202071  11.2880673010023  10.8631297240559  10.361905180101  9.93944940849568  9.49932316123333  9.10105039129286  8.77559583403619  8.47589951462709  8.23961908727907  7.97369548695164  7.77341294390762  7.58811002596642  7.43464348818395  7.620547330425  8.03689808015181  None
2   Afganistán          AFG  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN            31.921            31.349            30.845            30.359            29.867            29.389            28.872            28.396            27.895            27.398            26.875           26.364            25.845            25.297          

In [83]:
data_07 = data_07.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_mort_in_year)")

In [84]:
data_07.sample(False, 100/data_07.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                          Indicator Name  Indicator Code  Year ratio_mort_in_year
0         None          AFE  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN  1993   15.6194055296085
1   Afganistán          AFG  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN  1998             13.565
2       Angola          AGO  Tasa de mortalidad en un año (por cada 1.000 personas)  SP.DYN.CDRT.IN  2013              9.607

In [85]:
data_07 = data_07.drop("Country Name","Indicator Name","Indicator Code")

# Población entre 0 y 14 años de edad (% del total)
Algunos metadatos están solo en inglés

Población entre 0 y 14 años como porcentaje de la población total. La población se basa en la definición de facto de población.

https://datos.bancomundial.org/indicador/SP.POP.0014.TO.ZS?view=chart

In [86]:
url_08 = "https://api.worldbank.org/v2/es/indicator/SP.POP.0014.TO.ZS?downloadformat=csv"


In [87]:
response_url_08 = requests.get(url_08)

In [88]:
if response_url_08.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_08.content), "r") as zip_url_08:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_08.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_08.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [89]:
# Ruta del archivo CSV

csv_url_08 = "dataset/API_SP.POP.0014.TO.ZS_DS2_es_csv_v2_5521643.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_08 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_08, "r") as file_csv_url_08:
    csv_reader_url_08 = csv.reader(file_csv_url_08)
    rows_url_08 = list(csv_reader_url_08)

# Eliminar las primeras 4 filas

rows_url_08 = rows_url_08[3:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_08.csv", "w", newline="") as file_csv_url_08:
    csv_writer_url_08 = csv.writer(file_csv_url_08)
    csv_writer_url_08.writerows(rows_url_08)

In [90]:
data_08 = spark.read.csv("dataset/data/data_08.csv", header=True)

In [91]:
data_08 = data_08.drop("_c66","_c67")

In [92]:
data_08.limit(3).pandas_api()


Country Name Country Code                                     Indicator Name     Indicator Code              1960              1961              1962              1963              1964              1965              1966              1967              1968              1969              1970              1971              1972              1973              1974              1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  43.1310430706124  42.9494185733995  42.8527323247916  42.6611570247934  42.3591594688668  41.9366637462467  41.4062961182652   40.725620154062  39.9492185201066  39.1934940165178  38.4280916650453  37.6186954341044   36.690708599876  35.6158037917646  34.4725334221815  33.3498035921634  32.1853168280427  31.0564462991434  30.1394602999773   29.423765442054  28.9240602892406  28.6966868167408  28.4832689017048  28.1802050782783  27.9012691957086   27.534973169143  27.0591606896659  26.5275407292475  26.0813131980445  26.0056815103219  26.5592780563524  27.6965696098078   28.102405561927  27.5241846323936  26.8469626090041  26.0090850097339  25.1488975911958  24.4795866011874  23.9784245154407   23.608504955852  23.2556312499299  22.8794636703955  22.5041130517209  22.1027820627609  21.6689206164241  21.3578103997545  21.2401876441455  21.1831134346555  21.1304543042573  21.0513849131153  20.7486433828476  20.2250019745676  19.7372480083046  19.3487558320373  19.0455045659015  18.7996067429202  18.5717214384812  18.3348587091053  18.0697706242362  17.7673391737323  17.3510219591029  16.7994067760814  None
1         None          AFE  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  44.2000161820481   44.285568588289  44.3819921563279  44.4950978851111  44.6098096523218  44.7288589221977  44.8547629154212  44.9735331734901  45.0839111744781  45.1920354584117  45.3102679138395  45.4285747989249  45.5395933572709  45.6181880061979  45.6522277977864  45.6767443704248  45.7160922613424   45.739127929765  45.7206612590142  45.7020960422987  45.6741830578868  45.6605096568793  45.6649575169904  45.6365197830601  45.5786886264196  45.5197481444781  45.4788747945669  45.4754018375813  45.5069949121705  45.5386650874645  45.5602375679936  45.5808496987614  45.6075275574049  45.6435242910124  45.5685297989557  45.4488521680713  45.4166582225268  45.3629893920904  45.2259728997999   45.042418977025  44.8507743046745  44.6461589317462  44.4445584713958  44.2581117437644  44.0855575941329  43.9346272237786  43.7995792106487  43.6755418260986  43.5661319791078  43.4605719989725  43.3412574781264  43.2077638965504  43.0727197145178  42.9336951964389  42.7773435923648  42.5941707727048  42.4240042900302  42.2552613358142  42.0527862927101  41.8320998283195  41.5981147490549  41.3627387766738  None
2   Afganistán          AFG  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  41.6271864684651  41.6953029189524  41.7691668785528  41.8853766782279  42.0593887198501  42.3141327571861  42.6201405335114  42.9187974461615  43.2083750435887  43.4894284281289  43.7592715650739  44.0151410825322  44.2605080602237  44.497719

In [93]:
data_08 = data_08.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_poblation_0_14_year)")

In [94]:
data_08.sample(False, 3/data_08.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                     Indicator Name     Indicator Code  Year ratio_poblation_0_14_year
0         None          AFE  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  1993          45.6435242910124
1   Cabo Verde          CPV  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  2016          28.6145368552162
2       Israel          ISR  Población entre 0 y 14 años de edad (% del total)  SP.POP.0014.TO.ZS  1988          31.3151266173891

In [95]:
data_08 = data_08.drop("Country Name","Indicator Name","Indicator Code")

# Población entre 15 y 64 años de edad (% del total)

La población entre 15 y 64 años es el porcentaje de la población total en el grupo etario de 15 a 64 años. La población se basa en la definición de facto de población.

https://datos.bancomundial.org/indicador/SP.POP.1564.TO.ZS?view=chart




In [96]:
url_09 ="https://api.worldbank.org/v2/es/indicator/SP.POP.1564.TO.ZS?downloadformat=csv"

In [97]:
response_url_09 = requests.get(url_09)

In [98]:
if response_url_09.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_09.content), "r") as zip_url_09:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_09.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_09.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [99]:
# Ruta del archivo CSV

csv_url_09 = "dataset/API_SP.POP.1564.TO.ZS_DS2_es_csv_v2_5489860.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_09 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_09, "r") as file_csv_url_09:
    csv_reader_url_09 = csv.reader(file_csv_url_09)
    rows_url_09 = list(csv_reader_url_09)

# Eliminar las primeras 4 filas

rows_url_09 = rows_url_09[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_09.csv", "w", newline="") as file_csv_url_09:
    csv_writer_url_09 = csv.writer(file_csv_url_09)
    csv_writer_url_09.writerows(rows_url_09)



In [100]:
data_09 = spark.read.csv("dataset/data/data_09.csv", header=True)

In [101]:
data_09 = data_09.drop("_c66","_c67")

In [102]:
data_09.limit(3).pandas_api()


Country Name Country Code                                      Indicator Name     Indicator Code              1960              1961              1962              1963              1964              1965              1966              1967              1968              1969              1970              1971              1972              1973              1974              1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  54.4956782888954  54.5887011521026  54.5856304855996  54.6742061765985  54.8734476386919  55.1814771654347  55.5772003946669  56.0864561544988  56.6457319175053  57.1582673183887  57.6637087291584  58.1886035381228  58.8500649896781  59.6853391279447  60.5938944650368  61.4875937775362  62.4038893655268  63.2777737104555  63.9743755871586  64.5187562493952  64.8759766487598  64.8997420684038  64.8947968819317  65.0715261529765  65.2876142393007  65.6332392443935  66.1014979938965  66.5903801396431  66.9842922984852  67.0882646623991  66.6861964800682  65.7845101968643  65.5378500245756   66.147042564953  66.8031936608643  67.5853341985724  68.3789364997419  68.9633267365438  69.3592555272361  69.6196484280567  69.8252544864816  69.9709451375833  70.0700580730217  70.1966537577804  70.3381993703195  70.3571012774785  70.2185520858937  70.0372983975121  69.8421364137312  69.6508486876588  69.6416736843679  69.7940526024801  69.8594183808876  69.7953926905132  69.6208274610499  69.3523247727981   69.027742682921  68.6812691758703  68.3205692633645  67.9625149728727  67.7120031524284  67.6570940475799  None
1         None          AFE  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  52.8274157784759  52.7604902756957  52.6826758354695  52.5865096608518  52.4855200146271  52.3766281889937  52.2577618164994  52.1421537435157  52.0315761082849  51.9218982595951    51.80144226831  51.6792373973474  51.5638756920615  51.4804645818136  51.4419733135378  51.4142213823217  51.3699940717839  51.3413341806288  51.3525439741294  51.3619443024619  51.3868750841146  51.4008466283297  51.3955325823119  51.4281247559813   51.491572512832  51.5528417981183   51.597103219366  51.6009536893046  51.5738965551601  51.5522338123665  51.5369286027593  51.5186468555464  51.4926682415313  51.4607031320348  51.5444172479884  51.6742761687718  51.7121942522526   51.770112526495  51.9131917655101  52.1009742276052   52.292565736414  52.4952894128363  52.6952781133297  52.8812795012001  53.0530316413024  53.2014025490894  53.3335669231418  53.4548876185475   53.561747990582  53.6613407718808  53.7671315110531  53.8800733974156  53.9908273606468  54.1048190498773  54.2343884361198  54.3903502447806  54.5306781585856  54.6660704907447    54.83426630198  55.0215384291933  55.2351937516876  55.4719259144227  None
2   Afganistán          AFG  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  55.5397840942487  55.4870229598163  55.4317786177159  55.3356554098572   55.181682160916  54.9466202662646  54.6583323341661  54.3749968781312  54.0996271436168  53.8326614172133  53.5777064042251  53.3352829471189  53.1029186650456  52.87

In [103]:
data_09 = data_09.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_poblation_15_64_year)")

In [104]:
data_09.sample(False, 3/data_09.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                      Indicator Name     Indicator Code  Year ratio_poblation_15_64_year
0         None          AFE  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  1993           51.4607031320348
1   Cabo Verde          CPV  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  2016           66.1595878899237
2       Israel          ISR  Población entre 15 y 64 años de edad (% del total)  SP.POP.1564.TO.ZS  1988            59.200982670532

In [105]:
data_09 = data_09.drop("Country Name","Indicator Name","Indicator Code")

# Población de 65 años de edad y más (% del total)


Población mayor de 65 años como porcentaje de la población total. Población se basa en la definición establecida de dicho término, que contabiliza a todos los residentes independientemente de su situación legal o de ciudadanía, excluyendo a los refugiados no asentados permanentemente en el país de acogida, a quienes se les considera usualmente parte de la población de sus respectivos países de origen.

https://datos.bancomundial.org/indicador/SP.POP.65UP.TO.ZS?view=chart

In [106]:
url_10 ="https://api.worldbank.org/v2/es/indicator/SP.POP.65UP.TO.ZS?downloadformat=csv"

In [107]:
response_url_10 = requests.get(url_10)

In [108]:
if response_url_10.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_10.content), "r") as zip_url_10:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_10.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_10.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [109]:
# Ruta del archivo CSV

csv_url_10 = "dataset/API_SP.POP.65UP.TO.ZS_DS2_es_csv_v2_5455592.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_10 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_10, "r") as file_csv_url_10:
    csv_reader_url_10 = csv.reader(file_csv_url_10)
    rows_url_10 = list(csv_reader_url_10)

# Eliminar las primeras 4 filas
rows_url_10 = rows_url_10[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_10.csv", "w", newline="") as file_csv_url_10:
    csv_writer_url_10 = csv.writer(file_csv_url_10)
    csv_writer_url_10.writerows(rows_url_10)

In [110]:
data_10 = spark.read.csv("dataset/data/data_10.csv", header=True)

In [111]:
data_10 = data_10.drop("_c66","_c67")

In [112]:
data_10.limit(3).pandas_api()


Country Name Country Code                                    Indicator Name     Indicator Code              1960              1961              1962              1963              1964              1965              1966              1967              1968              1969              1970              1971              1972              1973              1974              1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021  2022
0        Aruba          ABW  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  2.37327864049224  2.46188027449786  2.56163718960879  2.66463679860809  2.76739289244124   2.8818590883186  3.01650348706791  3.18792369143923  3.40504956238807  3.64823866509354  3.90819960579632  4.19270102777282  4.45922641044592  4.69885708029074  4.93357211278164  5.16260263030034  5.41079380643053  5.66577999040112  5.88616411286404  6.05747830855078  6.19996306199963  6.40357111485542  6.62193421636352  6.74826876874519  6.81111656499077   6.8317875864636  6.83934131643766  6.88207913110939  6.93439450347031  6.90605382727899  6.75452546357943  6.51892019332783  6.35974441349736   6.3287728026534  6.34984373013164  6.40558079169371  6.47216590906229  6.55708666226881  6.66231995732322  6.77184661609127  6.91911426358851  7.14959119202121  7.42582887525741  7.70056417945869  7.99288001325643  8.28508832276706  8.54126026996073  8.77958816783246  9.02740928201151   9.2977663992259  9.60968293278454  9.98094542295237  10.4033336108078  10.8558514774495  11.3336679730486  11.8480684842817  12.4005358785978  12.9838721150244  13.6096601123993   14.270145853395  14.9369748884688  15.5434991763386  None
1         None          AFE  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  2.97256803947601  2.95394113601538  2.93533200820256  2.91839245403704  2.90467033305114  2.89451288880863  2.88747526807938  2.88431308299416  2.88451271723696  2.88606628199314  2.88828981785057  2.89218780372774  2.89653095066763  2.90134741198854   2.9057988886758  2.90903424725347  2.91391366687369  2.91953788960614  2.92679476685641  2.93595965523942  2.93894185799866  2.93864371479093   2.9395099006977   2.9353554609586  2.92973886074844  2.92741005740368  2.92402198606717  2.92364447311404  2.91910853266942  2.90910110016893  2.90283382924712  2.90050344569217  2.89980420106374  2.89577257695287  2.88705295305594   2.8768716631569  2.87114752522055  2.86689808141456  2.86083533469004  2.85660679536975  2.85665995891157  2.85855165541747  2.86016341527444  2.86060875503548  2.86141076456476  2.86397022713204  2.86685386620957  2.86957055535392  2.87212003031023  2.87808722914673  2.89161101082053  2.91216270603393  2.93645292483533   2.9614857536838  2.98826797151546  3.01547898251469  3.04531755138417  3.07866817344104  3.11294740530993   3.1463617424872  3.16669149925752  3.16533530890346  None
2   Afganistán          AFG  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  2.83302943728627  2.81767412123129  2.79905450373125  2.77896791191492  2.75892911923386  2.73924697654934  2.72152713232255  2.70620567570727  2.69199781279447  2.67791015465783    2.663022030701  2.64957597034893  2.63657327473071  2.62251416366

In [113]:
data_10 = data_10.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_poblation_more_64_year)")

In [114]:
data_10.sample(False, 3/data_09.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                    Indicator Name     Indicator Code  Year ratio_poblation_more_64_year
0         None          AFE  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  1993             2.89577257695287
1   Cabo Verde          CPV  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  2016             5.22587525486014
2       Israel          ISR  Población de 65 años de edad y más (% del total)  SP.POP.65UP.TO.ZS  1988             9.48389071207895

In [115]:
data_10 = data_10.drop("Country Name","Indicator Name","Indicator Code")

# Crecimiento de la población (% anual)

Annual population growth rate for year t is the exponential rate of growth of midyear population from year t-1 to t, expressed as a percentage . Population is based on the de facto definition of population, which counts all residents regardless of legal status or citizenship.

https://datos.bancomundial.org/indicador/SP.POP.GROW?view=chart


In [116]:
url_11 ="https://api.worldbank.org/v2/es/indicator/SP.POP.GROW?downloadformat=csv"

In [117]:
response_url_11 = requests.get(url_11)

In [118]:
if response_url_11.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_11.content), "r") as zip_url_11:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_11.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_11.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [119]:
# Ruta del archivo CSV

csv_url_11 = "dataset/API_SP.POP.GROW_DS2_es_csv_v2_5457838.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_11 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_11, "r") as file_csv_url_11:
    csv_reader_url_11 = csv.reader(file_csv_url_11)
    rows_url_11 = list(csv_reader_url_11)

# Eliminar las primeras 4 filas

rows_url_11 = rows_url_11[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_11.csv", "w", newline="") as file_csv_url_11:
    csv_writer_url_11 = csv.writer(file_csv_url_11)
    csv_writer_url_11.writerows(rows_url_11)

In [120]:
data_11 = spark.read.csv("dataset/data/data_11.csv", header=True)

In [121]:
data_11 = data_11.drop("_c66","_c67")

In [122]:
data_11.limit(3).pandas_api()


Country Name Country Code                         Indicator Name Indicator Code  1960              1961              1962              1963              1964              1965              1966               1967                 1968                1969                1970                1971                1972               1973              1974              1975               1976               1977               1978               1979               1980               1981               1982               1983               1984               1985                1986                1987                1988               1989               1990               1991              1992              1993              1994              1995              1996              1997              1998              1999              2000               2001              2002               2003               2004              2005              2006              2007              2008              2009              2010               2011               2012               2013               2014              2015               2016               2017               2018               2019               2020                 2021  2022
0        Aruba          ABW  Crecimiento de la población (% anual)    SP.POP.GROW  None  2.17905904113205  1.54857174393805   1.3893370634193  1.21572057526871  1.03284091886278  0.86218382371151  0.388846828315926  -0.0857193332764354  -0.237371850751822  -0.378263816895484  -0.491851536837525  0.0662865131222005  0.862803525331165  1.11062929159876  1.13796644720391  0.784201938574436  0.443510322777766  0.443171752006989  0.433152991694554  0.420043587504175  0.555730484303276  0.798540789902855  0.894334780580153  0.768049326467615  0.472593610369669   0.116251146540699  -0.159686242780767  -0.183255475179547  0.409531473341962   1.71290678473822   3.22241452912309  3.37286419184246  3.04193161032362  3.19602914606154  3.08406097784149  3.02578893630418  3.02735876796255  3.00480452491461  2.93441292558764  2.53923444445246   1.76875662143885  1.19471805545637  0.997395547284924  0.900989229759957  1.00307718391638  1.18156554573069  1.22771081489399  1.24139737678817  1.23323132057229  1.13154104048985  0.939355909629835  0.810230587788097  0.749301039347625  0.691615260101675  0.63795916173479  0.590062487333077  0.537295706145068  0.494795263046989  0.451969658863314  0.134255302359952  -0.0450446230906329  None
1         None          AFE  Crecimiento de la población (% anual)    SP.POP.GROW  None  2.66018011627118  2.73263311469827  2.75324827208996  2.80691497157682  2.84078687831499  2.81360949114521   2.83050527400394     2.88158851658864    2.89853202978372    2.91152613520499    2.91485587604232    2.87088142744661   2.90393831750268  2.98629564506714  2.98240909513056   2.96980990535009   2.88409601533984   3.01273283156685   3.14259397423467   2.96540623601945   3.01756402832467   3.13076216275165   3.01280081105612   2.93908811438321   2.97029294692479    3.02226383813252    3.06281660734588    2.91918186875581   2.84488762052256   2.91101286615705   2.79241035799646  2.63368225866559  2.65852640219622   2.6199545435715  2.62710702979302  2.69309772778222   2.5803841220214  2.51465393399148  2.55966248170103   2.5835792421522   2.58996060035651  2.60659802563768     2.617764283814   2.64496843028186  2.66624158730311  2.70252112091536  2.74710885116298  2.76581597609149  2.75574529766985  2.75792914247424   2.73818484544645   2.74040483423965   2.78015724354725    2.7749916302289  2.80258713657233   2.72815977582972   2.65567277976353   2.68837259222772   2.69113551540509   2.67818481620223     2.60747273691268  None
2   Afganistán          AFG  Crecimiento de la población (% anual)    SP.POP.GROW  None  1.92595161110872  2.01487886339461  2.07899662655224  2.13965084372647  2.21600692082533  2.25352401228011   2.29263788976095     2.34735064349307    2.37891654743303    2.43318262480668    2.4153

In [123]:
data_11 = data_11.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_population_growth)")

In [124]:
data_11.sample(False, 3/data_11.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                         Indicator Name Indicator Code  Year ratio_population_growth
0         None          AFE  Crecimiento de la población (% anual)    SP.POP.GROW  1993        2.65852640219622
1   Cabo Verde          CPV  Crecimiento de la población (% anual)    SP.POP.GROW  2016        1.12160806985756
2       Israel          ISR  Crecimiento de la población (% anual)    SP.POP.GROW  1988        1.65705755029368

In [125]:
data_11 = data_11.drop("Country Name","Indicator Name","Indicator Code")

# Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)


Mortality from CVD, cancer, diabetes or CRD is the percent of 30-year-old-people who would die before their 70th birthday from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease, assuming that s/he would experience current mortality rates at every age and s/he would not die from any other cause of death (e.g., injuries or HIV/AIDS).

https://datos.bancomundial.org/indicador/SH.DYN.NCOM.ZS?view=chart

In [126]:
url_12 ="https://api.worldbank.org/v2/es/indicator/SH.DYN.NCOM.ZS?downloadformat=csv"

In [127]:
response_url_12 = requests.get(url_12)

In [128]:
if response_url_12.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_12.content), "r") as zip_url_12:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_12.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_12.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [129]:
# Ruta del archivo CSV

csv_url_12 = "dataset/API_SH.DYN.NCOM.ZS_DS2_es_csv_v2_5476465.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_12 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_12, "r") as file_csv_url_12:
    csv_reader_url_12 = csv.reader(file_csv_url_12)
    rows_url_12 = list(csv_reader_url_12)

# Eliminar las primeras 4 filas

rows_url_12 = rows_url_12[3:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_12.csv", "w", newline="") as file_csv_url_12:
    csv_writer_url_12 = csv.writer(file_csv_url_12)
    csv_writer_url_12.writerows(rows_url_12)

In [130]:
data_12 = spark.read.csv("dataset/data/data_12.csv", header=True)

In [131]:
data_12 = data_12.drop("_c66","_c67")

In [132]:
data_12.limit(3).pandas_api()


Country Name Country Code                                                                                                                                               Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010             2011              2012              2013              2014              2015              2016              2017              2018              2019  2020  2021  2022
0        Aruba          ABW  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None              None              None              None              None              None              None              None              None             None              None              None              None              None              None              None              None              None  None  None  None
1         None          AFE  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  28.2733800536648  27.6771368480359  27.3046489815554  27.0930191887463  26.7782522823306  26.5421519153433  26.3391682496035  26.0046378067277  25.8937010959456  25.6081986661493  25.1987976042742  24.759162211563  24.2751403652027  23.9400712460361  23.6305671729774  23.4504709976819  23.2102460795151  22.8814394017632  22.5315597289214  22.2638109969202  None  None  None
2   Afganistán          AFG  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              41.7              42.1              42.1              41.5              41.4              40.9              40.5              40.1              39.4              38.4              37.8             37.1              36.6              36.2              35.7              35.6              35.6              35.5              35.4              35.3  None  None  None

In [133]:
data_12 = data_12.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_mortality_CVD_cancer_diabetes_CRD)")

In [134]:
data_12.sample(False, 3/data_12.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                                                                                                                               Indicator Name  Indicator Code  Year ratio_mortality_CVD_cancer_diabetes_CRD
0         None          AFE  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  1993                                    None
1   Cabo Verde          CPV  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  2016                                    15.1
2       Israel          ISR  Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)  SH.DYN.NCOM.ZS  1988                                    None

In [135]:
data_12 = data_12.drop("Country Name","Indicator Name","Indicator Code")

# Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)
Algunos metadatos están solo en inglés

Cause of death refers to the share of all deaths for all ages by underlying causes. Communicable diseases and maternal, prenatal and nutrition conditions include infectious and parasitic diseases, respiratory infections, and nutritional deficiencies such as underweight and stunting.

https://datos.bancomundial.org/indicador/SH.DTH.COMM.ZS?view=chart


In [136]:
url_13 ="https://api.worldbank.org/v2/es/indicator/SH.DTH.COMM.ZS?downloadformat=csv"

In [137]:
response_url_13 = requests.get(url_13)

In [138]:
if response_url_13.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_13.content), "r") as zip_url_13:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_13.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_13.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [139]:
# Ruta del archivo CSV

csv_url_13 = "dataset/API_SH.DTH.COMM.ZS_DS2_es_csv_v2_5494256.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_13 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_13, "r") as file_csv_url_13:
    csv_reader_url_13 = csv.reader(file_csv_url_13)
    rows_url_13 = list(csv_reader_url_13)

# Eliminar las primeras 4 filas
rows_url_13 = rows_url_13[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_13.csv", "w", newline="") as file_csv_url_13:
    csv_writer_url_13 = csv.writer(file_csv_url_13)
    csv_writer_url_13.writerows(rows_url_13)

In [140]:
data_13 = spark.read.csv("dataset/data/data_13.csv", header=True)

In [141]:
data_13 = data_13.drop("_c66","_c67")

In [142]:
data_13.limit(3).pandas_api()

Country Name Country Code                                                                                                                Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999              2000  2001  2002  2003  2004  2005  2006  2007  2008  2009              2010  2011  2012  2013  2014              2015  2016  2017  2018              2019  2020  2021  2022
0        Aruba          ABW  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None  None  None  None  None  None  None  None  None  None              None  None  None  None  None              None  None  None  None              None  None  None  None
1         None          AFE  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None   67.042887053288  None  None  None  None  None  None  None  None  None  59.1548866588657  None  None  None  None   52.911433522748  None  None  None  49.0198435796198  None  None  None
2   Afganistán          AFG  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  59.8146503168484  None  None  None  None  None  None  None  None  None  48.1499135188659  None  None  None  None  39.3909806318543  None  None  None  32.9304513537869  None  None  None

In [143]:
data_13 = data_13.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, ratio_infectious_diseases)")

In [144]:
data_13.sample(False, 3/data_13.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                                                                                                Indicator Name  Indicator Code  Year ratio_infectious_diseases
0         None          AFE  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  1993                      None
1   Cabo Verde          CPV  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  2016                      None
2       Israel          ISR  Causa de muerte, por enfermedades transmisibles y condiciones maternales, prenatales y en materia de nutrición (% del total)  SH.DTH.COMM.ZS  1988                      None

In [145]:
data_13 = data_13.drop("Country Name","Indicator Name","Indicator Code")

# Prevalencia de desnutrición (% de la población)

La población ubicada por debajo del nivel mínimo de consumo alimenticio de energía (indicador también conocido como prevalencia de desnutrición) muestra el porcentaje de la población cuya ingesta de alimentos no alcanza para satisfacer sus requisitos alimenticios de energía de manera continua. Los datos mostrados como 5 muestran una prevalencia de desnutrición menor del 5%

https://datos.bancomundial.org/indicador/SN.ITK.DEFC.ZS?view=chart

In [146]:
url_14 ="https://api.worldbank.org/v2/es/indicator/SN.ITK.DEFC.ZS?downloadformat=csv"

In [147]:
response_url_14 = requests.get(url_14)

In [148]:
if response_url_14.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_14.content), "r") as zip_url_14:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_14.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_14.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [149]:
# Ruta del archivo CSV

csv_url_14 = "dataset/API_SN.ITK.DEFC.ZS_DS2_es_csv_v2_5465519.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_14 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_14, "r") as file_csv_url_14:
    csv_reader_url_14 = csv.reader(file_csv_url_14)
    rows_url_14 = list(csv_reader_url_14)

# Eliminar las primeras 4 filas

rows_url_14 = rows_url_14[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_14.csv", "w", newline="") as file_csv_url_14:
    csv_writer_url_14 = csv.writer(file_csv_url_14)
    csv_writer_url_14.writerows(rows_url_14)

In [150]:
data_14 = spark.read.csv("dataset/data/data_14.csv", header=True)

In [151]:
data_14 = data_14.drop("_c66","_c67")

In [152]:
data_14.limit(3).pandas_api()

Country Name Country Code                                   Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999  2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018             2019              2020  2021  2022
0        Aruba          ABW  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None             None              None  None  None
1         None          AFE  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  33.0643933560435  32.6428707376993  32.2679936739954  31.0352017834494  29.6833670127598  28.5035112785995  28.0573791199456  27.4878580806953  25.7297057710846  23.9532905698366  22.2924321233599  21.7180622248758  21.4042727579889  21.4441615610215  21.9518481417626  22.4664366753986  23.0749593317083  23.6542422358629  25.128371989046  26.5420142899351  None  None
2   Afganistán          AFG  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              47.8              45.6              40.6                38              36.1              33.3              29.8              26.5              23.3              21.2              20.2              21.1              20.7              20.7              21.5              22.2                23                24             26.9              29.8  None  None

In [153]:
data_14 = data_14.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_prev_malnutrition)")

In [154]:
data_14.sample(False, 3/data_14.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                   Indicator Name  Indicator Code  Year porc_prev_malnutrition
0         None          AFE  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  1993                   None
1   Cabo Verde          CPV  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  2016                     17
2       Israel          ISR  Prevalencia de desnutrición (% de la población)  SN.ITK.DEFC.ZS  1988                   None

In [155]:
data_14 = data_14.drop("Country Name","Indicator Name","Indicator Code")

# Médicos (por cada 1.000 personas)

Los médicos incluyen a profesionales de cabecera y especialistas.

https://datos.bancomundial.org/indicador/SH.MED.PHYS.ZS?view=chart

In [156]:
url_15 ="https://api.worldbank.org/v2/es/indicator/SH.MED.PHYS.ZS?downloadformat=csv"

In [157]:
response_url_15 = requests.get(url_15)

In [158]:
if response_url_15.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_15.content), "r") as zip_url_15:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_15.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_15.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [159]:
# Ruta del archivo CSV

csv_url_15 = "dataset/API_SH.MED.PHYS.ZS_DS2_es_csv_v2_5465722.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_15 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_15, "r") as file_csv_url_15:
    csv_reader_url_15 = csv.reader(file_csv_url_15)
    rows_url_15 = list(csv_reader_url_15)

# Eliminar las primeras 4 filas

rows_url_15 = rows_url_15[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/dataset_15.csv", "w", newline="") as file_csv_url_15:
    csv_writer_url_15 = csv.writer(file_csv_url_15)
    csv_writer_url_15.writerows(rows_url_15)

In [160]:
data_15 = spark.read.csv("dataset/data/dataset_15.csv", header=True)

In [161]:
data_15 = data_15.drop("_c66","_c67")

In [162]:
data_15.limit(3).pandas_api()

Country Name Country Code                     Indicator Name  Indicator Code   1960  1961  1962  1963  1964   1965  1966  1967  1968  1969   1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980   1981  1982  1983  1984  1985   1986   1987  1988   1989               1990  1991  1992   1993  1994  1995  1996  1997  1998  1999  2000    2001  2002  2003  2004  2005    2006    2007    2008    2009               2010    2011    2012    2013    2014   2015    2016    2017               2018    2019    2020  2021  2022
0        Aruba          ABW  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None               None  None  None   None  None  1.12  None  None  None  None  None    None  None  None  None  None    None    None    None    None               None    None    None    None    None   None    None    None               None    None    None  None  None
1         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None  0.160298794326942  None  None   None  None  None  None  None  None  None  None    None  None  None  None  None    None    None    None    None  0.103359172473203    None    None    None    None   None    None    None  0.229967326754284    None    None  None  None
2   Afganistán          AFG  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  0.035  None  None  None  None  0.063  None  None  None  None  0.065  None  None  None  None  None  None  None  None  None  None  0.077  None  None  None  None  0.183  0.179  None  0.129              0.109  None  None  0.143  None  None  None  0.11  None  None  None  0.1899  None  None  None  None  0.1596  0.1743  0.1744  0.2126             0.2365  0.2518  0.2414  0.2846  0.2983  0.285  0.2782  0.2409             0.2966  0.2125  0.2538  None  None

In [163]:
data_15 = data_15.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, doctor_1000_people)")

In [164]:
data_15.sample(False, 3/data_15.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                     Indicator Name  Indicator Code  Year doctor_1000_people
0         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1993               None
1   Cabo Verde          CPV  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  2016             0.7682
2       Israel          ISR  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1988               None

In [165]:
data_15 = data_15.drop("Country Name","Indicator Name","Indicator Code")

# Camas hospitalarias (por cada 1.000 personas)

Las camas hospitalarias se refiere a la cantidad de camas disponibles para internaciones en hospitales públicos, privados, generales y especializados, y centros de rehabilitación. En la mayoría de los casos se incluyen tanto las camas para atención de agudos como las camas para atención de enfermos crónicos.

https://datos.bancomundial.org/indicador/SH.MED.BEDS.ZS?view=chart

In [166]:
url_16 ="https://api.worldbank.org/v2/es/indicator/SH.MED.BEDS.ZS?downloadformat=csv"

In [167]:
response_url_16 = requests.get(url_16)

In [168]:
if response_url_16.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_16.content), "r") as zip_url_16:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_16.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_16.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [169]:
# Ruta del archivo CSV

csv_url_16 = "dataset/API_SH.MED.BEDS.ZS_DS2_es_csv_v2_5465852.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_16 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_16, "r") as file_csv_url_16:
    csv_reader_url_16 = csv.reader(file_csv_url_16)
    rows_url_16 = list(csv_reader_url_16)

# Eliminar las primeras 4 filas

rows_url_16 = rows_url_16[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_16.csv", "w", newline="") as file_csv_url_16:
    csv_writer_url_16 = csv.writer(file_csv_url_16)
    csv_writer_url_16.writerows(rows_url_16)

In [170]:
data_16 = spark.read.csv("dataset/data/dataset_15.csv", header=True)

In [171]:
data_16 = data_16.drop("_c66","_c67")

In [172]:
data_16.limit(3).pandas_api()

Country Name Country Code                     Indicator Name  Indicator Code   1960  1961  1962  1963  1964   1965  1966  1967  1968  1969   1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980   1981  1982  1983  1984  1985   1986   1987  1988   1989               1990  1991  1992   1993  1994  1995  1996  1997  1998  1999  2000    2001  2002  2003  2004  2005    2006    2007    2008    2009               2010    2011    2012    2013    2014   2015    2016    2017               2018    2019    2020  2021  2022
0        Aruba          ABW  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None               None  None  None   None  None  1.12  None  None  None  None  None    None  None  None  None  None    None    None    None    None               None    None    None    None    None   None    None    None               None    None    None  None  None
1         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None  0.160298794326942  None  None   None  None  None  None  None  None  None  None    None  None  None  None  None    None    None    None    None  0.103359172473203    None    None    None    None   None    None    None  0.229967326754284    None    None  None  None
2   Afganistán          AFG  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  0.035  None  None  None  None  0.063  None  None  None  None  0.065  None  None  None  None  None  None  None  None  None  None  0.077  None  None  None  None  0.183  0.179  None  0.129              0.109  None  None  0.143  None  None  None  0.11  None  None  None  0.1899  None  None  None  None  0.1596  0.1743  0.1744  0.2126             0.2365  0.2518  0.2414  0.2846  0.2983  0.285  0.2782  0.2409             0.2966  0.2125  0.2538  None  None

In [173]:
data_16 = data_16.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, bed_1000_people)")

In [174]:
data_16.sample(False, 3/data_16.count(), seed=150).limit(3).pandas_api()

Country Name Country Code                     Indicator Name  Indicator Code  Year bed_1000_people
0         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1993            None
1   Cabo Verde          CPV  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  2016          0.7682
2       Israel          ISR  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1988            None

In [175]:
data_16 = data_16.drop("Country Name","Indicator Name","Indicator Code")
data_16.columns

['Country Code', 'Year', 'bed_1000_people']

# People using at least basic drinking water services (% of population)

The percentage of people using at least basic water services. This indicator encompasses both people using basic water services as well as those using safely managed water services. Basic drinking water services is defined as drinking water from an improved source, provided collection time is not more than 30 minutes for a round trip. Improved water sources include piped water, boreholes or tubewells, protected dug wells, protected springs, and packaged or delivered water.

https://datos.bancomundial.org/indicador/SH.H2O.BASW.ZS?view=chart

In [176]:
url_17 ="https://api.worldbank.org/v2/es/indicator/SH.H2O.BASW.ZS?downloadformat=csv"

In [177]:
response_url_17 = requests.get(url_17)

In [178]:
if response_url_17.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_17.content), "r") as zip_url_17:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_17.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_17.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [179]:
# Ruta del archivo CSV
csv_url_17 = "dataset/API_SH.H2O.BASW.ZS_DS2_es_csv_v2_5522723.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_17 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_17, "r") as file_csv_url_17:
    csv_reader_url_17 = csv.reader(file_csv_url_17)
    rows_url_17 = list(csv_reader_url_17)

# Eliminar las primeras 4 filas

rows_url_17 = rows_url_17[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_17.csv", "w", newline="") as file_csv_url_17:
    csv_writer_url_17 = csv.writer(file_csv_url_17)
    csv_writer_url_17.writerows(rows_url_17)

In [180]:
data_17 = spark.read.csv("dataset/data/dataset_15.csv", header=True)

In [181]:
data_17 = data_17.drop("_c66","_c67")

In [182]:
data_17.limit(3).pandas_api()

Country Name Country Code                     Indicator Name  Indicator Code   1960  1961  1962  1963  1964   1965  1966  1967  1968  1969   1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980   1981  1982  1983  1984  1985   1986   1987  1988   1989               1990  1991  1992   1993  1994  1995  1996  1997  1998  1999  2000    2001  2002  2003  2004  2005    2006    2007    2008    2009               2010    2011    2012    2013    2014   2015    2016    2017               2018    2019    2020  2021  2022
0        Aruba          ABW  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None               None  None  None   None  None  1.12  None  None  None  None  None    None  None  None  None  None    None    None    None    None               None    None    None    None    None   None    None    None               None    None    None  None  None
1         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS   None  None  None  None  None   None  None  None  None  None   None  None  None  None  None  None  None  None  None  None  None   None  None  None  None  None   None   None  None   None  0.160298794326942  None  None   None  None  None  None  None  None  None  None    None  None  None  None  None    None    None    None    None  0.103359172473203    None    None    None    None   None    None    None  0.229967326754284    None    None  None  None
2   Afganistán          AFG  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  0.035  None  None  None  None  0.063  None  None  None  None  0.065  None  None  None  None  None  None  None  None  None  None  0.077  None  None  None  None  0.183  0.179  None  0.129              0.109  None  None  0.143  None  None  None  0.11  None  None  None  0.1899  None  None  None  None  0.1596  0.1743  0.1744  0.2126             0.2365  0.2518  0.2414  0.2846  0.2983  0.285  0.2782  0.2409             0.2966  0.2125  0.2538  None  None

In [183]:
data_17 = data_17.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_water_service)")

In [184]:
data_17.sample(False, 3/data_16.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                     Indicator Name  Indicator Code  Year porc_water_service
0         None          AFE  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1993               None
1   Cabo Verde          CPV  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  2016             0.7682
2       Israel          ISR  Médicos (por cada 1.000 personas)  SH.MED.PHYS.ZS  1988               None

In [185]:
data_17 = data_17.drop("Country Name","Indicator Name","Indicator Code")
data_17.columns

['Country Code', 'Year', 'porc_water_service']

# Personas que utilizan al menos servicios básicos de sanitarios (% de la población)
People using at least basic sanitation services (% of population)

The percentage of people using at least basic sanitation services, that is, improved sanitation facilities that are not shared with other households. This indicator encompasses both people using basic sanitation services as well as those using safely managed sanitation services. Improved sanitation facilities include flush/pour flush to piped sewer systems, septic tanks or pit latrines; ventilated improved pit latrines, compositing toilets or pit latrines with slabs.

https://datos.bancomundial.org/indicador/SH.STA.BASS.ZS?view=chart

In [186]:
url_18 ="https://api.worldbank.org/v2/es/indicator/SH.STA.BASS.ZS?downloadformat=csv"

In [187]:
response_url_18 = requests.get(url_18)

In [188]:
if response_url_18.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_18.content), "r") as zip_url_18:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_18.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_18.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [189]:
# Ruta del archivo CSV
csv_url_18 = "dataset/API_SH.STA.BASS.ZS_DS2_es_csv_v2_5504500.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_18 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_18, "r") as file_csv_url_18:
    csv_reader_url_18 = csv.reader(file_csv_url_18)
    rows_url_18 = list(csv_reader_url_18)

# Eliminar las primeras 4 filas

rows_url_18 = rows_url_18[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_18.csv", "w", newline="") as file_csv_url_18:
    csv_writer_url_18 = csv.writer(file_csv_url_18)
    csv_writer_url_18.writerows(rows_url_18)

In [190]:
data_18 = spark.read.csv("dataset/data/data_18.csv", header=True)

In [191]:
data_18 = data_18.drop("_c66","_c67")

In [192]:
data_18.limit(3).pandas_api()

Country Name Country Code                                                     Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010            2011             2012              2013              2014              2015              2016              2017              2018              2019              2020  2021  2022
0        Aruba          ABW  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None       98.32020997       98.25548257       98.19075516       98.12602776       98.06130035       97.99657295       97.93184555       97.86711814       97.80239074       97.73766333       97.67293593     97.60820852      97.54348112       97.54348112       97.54348112       97.54348112       97.54348112              None              None              None              None  None  None
1         None          AFE  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  22.8211065402296  23.2114986094774  23.6005186330166  23.9893097400314  24.3751904507651  24.8267807588928  25.2590283093078  25.6688533218626  26.1318641507104  26.6044167362961  27.0751579646983  27.14586376719  27.607319729902  28.0768927689278  28.5678982360644  29.0837989288035  29.5521388922933  30.0182843340688  30.2964096753337  30.6558737186025  30.8359427084217  None  None
2   Afganistán          AFG  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None       21.87080211       21.88130402       23.27419282       24.66743482       26.06769471       27.47541582        28.8840397       30.37464076       31.87385375       33.38186228       34.89884717      36.4251488      37.96080939       39.50601764       41.06096635       42.62602284       44.20121748       45.78912703       47.39066209       49.00616952       50.49964687  None  None

In [193]:
data_18 = data_18.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_sanitation_services)")

In [194]:
data_18.sample(False, 3/data_18.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                                     Indicator Name  Indicator Code  Year porc_sanitation_services
0         None          AFE  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  1993                     None
1   Cabo Verde          CPV  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  2016              72.27341972
2       Israel          ISR  People using at least basic sanitation services (% of population)  SH.STA.BASS.ZS  1988                     None

In [195]:
data_18 = data_18.drop("Country Name","Indicator Name","Indicator Code")
data_18.columns

['Country Code', 'Year', 'porc_sanitation_services']

# Acceso a la electricidad (% de población)

Acceso a la electricidad es el porcentaje de población con acceso a la electricidad. Los datos de electrificación se recopilan de la industria, las encuestas

https://datos.bancomundial.org/indicador/EG.ELC.ACCS.ZS?view=chart

In [196]:
url_19 ="https://api.worldbank.org/v2/es/indicator/EG.ELC.ACCS.ZS?downloadformat=csv"

In [197]:
response_url_19 = requests.get(url_19)

In [198]:
if response_url_19.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_19.content), "r") as zip_url_19:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_19.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_19.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [199]:
# Ruta del archivo CSV
csv_url_19 = "dataset/API_EG.ELC.ACCS.ZS_DS2_es_csv_v2_5467323.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_19 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_19, "r") as file_csv_url_19:
    csv_reader_url_19 = csv.reader(file_csv_url_19)
    rows_url_19 = list(csv_reader_url_19)

# Eliminar las primeras 4 filas

rows_url_19 = rows_url_19[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_19.csv", "w", newline="") as file_csv_url_19:
    csv_writer_url_19 = csv.writer(file_csv_url_19)
    csv_writer_url_19.writerows(rows_url_19)

In [200]:
data_19 = spark.read.csv("dataset/data/data_19.csv", header=True)

In [201]:
data_19 = data_19.drop("_c66","_c67")

In [202]:
data_19.limit(3).pandas_api()

Country Name Country Code                             Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990              1991              1992              1993  1994  1995  1996  1997  1998  1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020  2021  2022
0        Aruba          ABW  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None   100  99.1536560058594  99.1971282958984  99.2399139404297   100   100   100   100   100   100   91.660400390625               100               100               100               100               100               100               100               100               100  93.3562927246094               100               100               100               100               100               100               100               100               100               100  None  None
1         None          AFE  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None  None  None  None  None  None  None  19.9579298951035  19.9862203237456  21.6015034178654  22.5483066365335  23.8158804627515  23.4811562468684  24.7499128916126  26.7515912769659   25.913958456892  26.1513599969695  27.4827342843522  28.9486279353382  31.6823177047106  31.6106918811145  31.8249495513156  33.7444046301052  38.7333520303803  40.0921632933888  42.8809766258104  44.0739121045303  45.6096042884373  None  None
2   Afganistán          AFG  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None  None  None  None  None  None  None  1.61359095573425  4.07457399368286  9.40915775299072  14.7385063171387  20.0649681091309  25.3908939361572  30.7186908721924  36.0510101318359  42.4000015258789  46.7400512695313  42.7000007629395  43.2220191955566  69.0999984741211  68.2906494140625              89.5              71.5  97.6999969482422  97.6999969482422  96.6161346435547  97.6999969482422  97.6999969482422  None  None

In [203]:
data_19 = data_19.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_electricity)")

In [204]:
data_19.sample(False, 3/data_19.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                             Indicator Name  Indicator Code  Year  porc_electricity
0         None          AFE  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  1993              None
1   Cabo Verde          CPV  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  2016  87.5449295043945
2       Israel          ISR  Acceso a la electricidad (% de población)  EG.ELC.ACCS.ZS  1988              None

In [205]:
data_19 = data_19.drop("Country Name","Indicator Name","Indicator Code")
data_19.columns

['Country Code', 'Year', 'porc_electricity']

# Personas que usan Internet (% de la población)

Internet users are individuals who have used the Internet (from any location) in the last 3 months. The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.

https://datos.bancomundial.org/indicador/IT.NET.USER.ZS?view=chart


In [206]:
url_20 ="https://api.worldbank.org/v2/es/indicator/IT.NET.USER.ZS?downloadformat=csv"

In [207]:
response_url_20 = requests.get(url_20)

In [208]:
if response_url_20.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_20.content), "r") as zip_url_20:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_20.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_20.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [209]:
# Ruta del archivo CSV
csv_url_20 = "dataset/API_IT.NET.USER.ZS_DS2_es_csv_v2_5458342.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_20 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_20, "r") as file_csv_url_20:
    csv_reader_url_20 = csv.reader(file_csv_url_20)
    rows_url_20 = list(csv_reader_url_20)

# Eliminar las primeras 4 filas

rows_url_20 = rows_url_20[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_20.csv", "w", newline="") as file_csv_url_20:
    csv_writer_url_20 = csv.writer(file_csv_url_20)
    csv_writer_url_20.writerows(rows_url_20)

In [210]:
data_20 = spark.read.csv("dataset/data/data_20.csv", header=True)

In [211]:
data_20 = data_20.drop("_c66","_c67")

In [212]:
data_20.limit(3).pandas_api()

Country Name Country Code                                  Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989 1990                 1991                 1992                1993                1994               1995               1996               1997               1998               1999              2000               2001             2002              2003              2004             2005              2006             2007              2008              2009              2010              2011              2012              2013              2014              2015             2016              2017              2018              2019              2020             2021  2022
0        Aruba          ABW  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None    0                    0                    0                   0                   0                  0        2.768382663               None               None        4.506179098       15.44282295               17.1             18.8              20.8                23             25.4                28             30.9                52                58                62                69                74              78.9             83.78       88.66122693      93.54245387             97.17              None              None              None             None  None
1         None          AFE  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None    0  0.00173230248880555  0.00504080836448765  0.0146294941425636  0.0317097336371475  0.086496310561846  0.112752212160974  0.213192402155205  0.386772416959473  0.569512051222246  0.77421755469409  0.961552005089705  1.1316294433033  1.43039835507724  1.71604667481381  1.8055231421271  2.05810176814815  2.7732738256208  2.69414978147317  3.20741617179148  5.29697361379769  6.80275022096875  8.53475011696143  10.1269454655226  12.2377157932034  14.4858491534531  15.753329822739  17.3109868719447  20.0630242595526  22.5895992519959  24.9884008681592  27.660653781351  None
2   Afganistán          AFG  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None    0                    0                    0                   0                   0                  0               None               None               None               None              None        0.004722568      0.004561395       0.087891253        0.10580903      1.224148084       2.107123645              1.9              1.84              3.55                 4                 5       5.454545455               5.9                 7              8.26               11              13.5              16.8              17.6              18.4             None  None

In [213]:
data_20 = data_20.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_internet)")

In [214]:
data_20.sample(False, 3/data_20.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                  Indicator Name  Indicator Code  Year       porc_internet
0         None          AFE  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  1993  0.0146294941425636
1   Cabo Verde          CPV  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  2016         50.32281787
2       Israel          ISR  Personas que usan Internet (% de la población)  IT.NET.USER.ZS  1988                None

In [215]:
data_20 = data_20.drop("Country Name","Indicator Name","Indicator Code")
data_20.columns

['Country Code', 'Year', 'porc_internet']

# Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)

Tasa de finalización de la educación primaria, total. Corresponde al número total de estudiantes que ingresan al último año de educación primaria, independientemente de su edad, expresado como porcentaje de la población total en edad oficial de ingresar a dicho grado. Este indicador también se denomina “tasa bruta de ingreso al último grado de primaria”. Esta tasa puede ser superior a 100% debido a niños mayores y menores de la edad oficial de ingreso que entran a la educación primaria de forma precoz o tardía y (o) que han repetido grado. 

https://datos.bancomundial.org/indicador/SE.PRM.CMPT.ZS?view=chart

In [216]:
url_21 ="https://api.worldbank.org/v2/es/indicator/SE.PRM.CMPT.ZS?downloadformat=csv"

In [217]:
response_url_21 = requests.get(url_21)

In [218]:
if response_url_21.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_21.content), "r") as zip_url_21:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_21.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_21.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [219]:
# Ruta del archivo CSV
csv_url_21 = "dataset/API_SE.PRM.CMPT.ZS_DS2_es_csv_v2_5478942.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_21 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_21, "r") as file_csv_url_21:
    csv_reader_url_21 = csv.reader(file_csv_url_21)
    rows_url_21 = list(csv_reader_url_21)

# Eliminar las primeras 4 filas
rows_url_21 = rows_url_21[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_21.csv", "w", newline="") as file_csv_url_21:
    csv_writer_url_21 = csv.writer(file_csv_url_21)
    csv_writer_url_21.writerows(rows_url_21)

In [220]:
data_21 = spark.read.csv("dataset/data/data_21.csv", header=True)

In [221]:
data_21 = data_21.drop("_c66","_c67")

In [222]:
data_21.limit(3).pandas_api()

Country Name Country Code                                                                                      Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970              1971             1972              1973              1974              1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985              1986              1987              1988              1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020  2021  2022
0        Aruba          ABW  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  None  None  None  None  None  None  None  None  None  None  None              None             None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None  101.619117736816  97.1428604125977  94.4040679931641   94.755241394043  90.2159194946289  90.5598983764648  88.1979675292969  93.1861190795898  95.5882415771484  95.1330337524414  96.2425765991211  94.7861022949219              None              None  96.5706405639648  101.180557250977              None              None              None              None              None              None  None  None
1         None          AFE  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  None  None  None  None  None  None  None  None  None  None  None  41.6629981994629  42.637508392334  43.3848991394043   44.429931640625  45.4001998901367  46.2538108825684  47.8404083251953  49.3253402709961  51.9174003601074  52.6832809448242   54.075008392334  55.2577514648438  55.7916107177734  58.8742713928223  55.2768516540527  54.8070411682129  53.6615295410156  52.3084907531738  50.6883811950684  51.5261611938477  52.1606483459473  51.1751708984375  52.1927604675293  50.7563018798828  50.8544692993164  51.0122604370117  50.9097900390625  50.0661392211914  50.3088989257813  50.4125289916992  52.4814109802246  54.2634696960449  55.9368896484375  57.2593002319336  59.5904998779297  61.8936386108398  63.5756912231445  66.9495315551758  70.1065673828125  69.5502166748047  68.3240737915039  68.0312576293945  67.6544494628906  67.5930099487305  67.2124328613281  67.4798126220703  68.6649017333984  70.1976623535156  70.9132766723633  71.9691619873047  None  None
2   Afganistán          AFG  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  None  None  None  None  None  None  None  None  None  None  None              None             None              None  16.6569995880127              None              None  17.8807601928711  19.6528396606445              None  26.3732395172119  32.7061195373535              None              None  17.8910293579102  18.5706005096436              None              None              None              None              None              None              None  27.5751705169678              Non

In [223]:
data_21 = data_21.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_primary_school_completion)")

In [224]:
data_21.sample(False, 3/data_20.count(), seed=150).limit(3).pandas_api()


Country Name Country Code                                                                                      Indicator Name  Indicator Code  Year porc_primary_school_completion
0         None          AFE  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  1993               52.1927604675293
1   Cabo Verde          CPV  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  2016               91.0370635986328
2       Israel          ISR  Tasa de finalización de la educación de nivel primario, total (% del grupo etario correspondiente)  SE.PRM.CMPT.ZS  1988                           None

In [225]:
data_21 = data_21.drop("Country Name","Indicator Name","Indicator Code")
data_21.columns

['Country Code', 'Year', 'porc_primary_school_completion']

# Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)

Lower secondary education completion rate is measured as the gross intake ratio to the last grade of lower secondary education (general and pre-vocational). It is calculated as the number of new entrants in the last grade of lower secondary education, regardless of age, divided by the population at the entrance age for the last grade of lower secondary education.

https://datos.bancomundial.org/indicador/SE.SEC.CMPT.LO.ZS?view=chart

In [226]:
url_22 ="https://api.worldbank.org/v2/es/indicator/SE.SEC.CMPT.LO.ZS?downloadformat=csv"

In [227]:
response_url_22 = requests.get(url_22)

In [228]:
if response_url_22.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_22.content), "r") as zip_url_22:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_22.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_22.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [229]:
# Ruta del archivo CSV

csv_url_22 = "dataset/API_SE.SEC.CMPT.LO.ZS_DS2_es_csv_v2_5473788.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_22 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_22, "r") as file_csv_url_22:
    csv_reader_url_22 = csv.reader(file_csv_url_22)
    rows_url_22 = list(csv_reader_url_22)

# Eliminar las primeras 4 filas
rows_url_22 = rows_url_22[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_22.csv", "w", newline="") as file_csv_url_22:
    csv_writer_url_22 = csv.writer(file_csv_url_22)
    csv_writer_url_22.writerows(rows_url_22)

In [230]:
data_22 = spark.read.csv("dataset/data/data_22.csv", header=True)

In [231]:
data_22 = data_22.drop("_c66","_c67")

In [232]:
data_22.limit(3).pandas_api()

Country Name Country Code                                                                                             Indicator Name     Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972              1973              1974  1975              1976              1977              1978              1979              1980              1981              1982              1983              1984              1985             1986              1987              1988              1989              1990              1991              1992              1993             1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020  2021  2022
0        Aruba          ABW  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None  None              None              None              None              None              None              None              None              None              None              None             None              None              None              None              None              None              None              None             None              None              None              None              None  93.6170196533203  94.8262481689453  98.2667694091797  97.1386566162109  89.5258331298828  87.5594787597656  87.5902786254883              None  89.0220794677734              None  96.1910934448242              None              None              None  101.545700073242  99.2572631835938              None              None              None              None              None              None  None  None
1         None          AFE  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None  None  24.3201103210449   24.511999130249  24.6560707092285  24.7215194702148  24.9958992004395  25.0259304046631  25.1622695922852  25.2455806732178  25.4166297912598  25.5515899658203  25.499080657959  25.5499801635742  25.7534408569336  25.9363708496094  25.7921600341797  25.8832893371582  25.7905902862549  26.5919990539551  26.948600769043  28.2395706176758  28.6701202392578  28.8856105804443  29.5753993988037  29.7661399841309  30.0601005554199  31.6068305969238  33.6173896789551  32.8705101013184  33.1322784423828  34.1036491394043  35.4262084960938  36.7879104614258  38.6221504211426  39.5837516784668  40.7856101989746  41.4754486083984  41.9407501220703  42.6220397949219  43.7790412902832  44.0443916320801  44.0727005004883  44.0504493713379  43.9863014221191  44.3850288391113  44.5507392883301  None  None
2   Afganistán          AFG  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  9.60499954223633  8.40972995758057  None              None  6.96337985992432  9.33279991149902              None  13.4953098297119  16.1467800140381  10.7438201904297              None  12.7984104156494  13.1899900436401             None              None              None              None              None              None              None  17.8644599914551             None              None              None              None              None              None              Non

In [233]:
data_22 = data_22.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_secundary_school_compl)")

In [234]:
data_22.sample(False, 3/data_22.count(), seed=150).limit(3).pandas_api()

Country Name Country Code                                                                                             Indicator Name     Indicator Code  Year porc_secundary_school_compl
0         None          AFE  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  1993            26.5919990539551
1   Cabo Verde          CPV  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  2016            79.1698989868164
2       Israel          ISR  Tasa de finalización del ciclo inferior de la educación secundaria, total (% del grupo etario pertinente)  SE.SEC.CMPT.LO.ZS  1988                        None

In [235]:
data_22 = data_22.drop("Country Name","Indicator Name","Indicator Code")
data_22.columns

['Country Code', 'Year', 'porc_secundary_school_compl']

# Tasa (%) de alfabetización de adultos (15+), total. 

Corresponde al porcentaje de la población mayor de 15 años que es capaz de leer y escribir, con entendimiento, una proposición simple y breve sobre su vida diaria. En general, el término “alfabetización” incluye también habilidades aritméticas, es decir, la capacidad de hacer cálculos matemáticos sencillos. Para calcular este indicador se divide el número de personas alfabetizadas mayores de 15 años por la población del correspondiente grupo etáreo y se multiplica por 100. 

https://datos.bancomundial.org/indicador/SE.ADT.LITR.ZS?view=chart


In [236]:
url_23 ="https://api.worldbank.org/v2/es/indicator/SE.ADT.LITR.ZS?downloadformat=csv"

In [237]:
response_url_23 = requests.get(url_23)

In [238]:
if response_url_23.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_23.content), "r") as zip_url_23:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_23.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_23.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [239]:
# Ruta del archivo CSV
csv_url_23 = "dataset/API_SE.ADT.LITR.ZS_DS2_es_csv_v2_5478959.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_23 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_23, "r") as file_csv_url_23:
    csv_reader_url_23 = csv.reader(file_csv_url_23)
    rows_url_23 = list(csv_reader_url_23)

# Eliminar las primeras 4 filas
rows_url_23 = rows_url_23[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_23.csv", "w", newline="") as file_csv_url_23:
    csv_writer_url_23 = csv.writer(file_csv_url_23)
    csv_writer_url_23.writerows(rows_url_23)

In [240]:
data_23 = spark.read.csv("dataset/data/data_23.csv", header=True)

In [241]:
data_23 = data_23.drop("_c66","_c67")

In [242]:
data_23.limit(3).pandas_api()

Country Name Country Code                                                             Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978              1979  1980  1981  1982  1983  1984  1985  1986              1987              1988             1989              1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004             2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020             2021  2022
0        Aruba          ABW  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None  None  None  None  None  None  None  None              None              None             None              None              None              None              None              None              None              None              None              None              None  97.2912521362305              None              None              None              None             None              None              None              None              None   96.822639465332              None              None              None              None              None              None              None  97.8074188232422              None  97.9899978637695             None  None
1         None          AFE  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None  None  None  None  None  None  None  None  55.7706718444824  56.0884208679199  56.601619720459  57.1959495544434  57.1739692687988  57.5963096618652  58.0278816223145  58.7079315185547  59.1105690002441  59.6861686706543  60.4427795410156  60.6150703430176  61.4292602539063  61.7969703674316  62.5126495361328  62.8830490112305  63.5342483520508  64.8186111450195  63.146110534668  61.2504501342773  61.9769897460938  62.4322204589844  63.0463714599609  63.6625785827637  65.8645401000977  67.3918228149414  68.4451065063477  68.8279724121094  69.2824935913086  70.0596008300781  69.9994506835938  70.4202499389648  71.5740509033203  71.8899078369141             None  None
2   Afganistán          AFG  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  18.1576805114746  None  None  None  None  None  None  None              None              None             None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None             None              None              None              None              None              None  31.4488506317139              None              None              None              None              None              None              None              None              None  37.266040802002  None

In [243]:
data_23 = data_23.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_literacy)")

In [244]:
data_23.sample(False, 3/data_23.count(), seed=150).limit(3).pandas_api()

Country Name Country Code                                                             Indicator Name  Indicator Code  Year     porc_literacy
0         None          AFE  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  1993  58.0278816223145
1   Cabo Verde          CPV  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  2016              None
2       Israel          ISR  Tasa de alfabetización, total de adultos (% de personas de 15 años o más)  SE.ADT.LITR.ZS  1988              None

In [245]:
data_23 = data_23.drop("Country Name","Indicator Name","Indicator Code")
data_23.columns

['Country Code', 'Year', 'porc_literacy']

# Desempleo, total (% de la población activa total) (estimación modelado OIT)

El desempleo es la proporción de la población activa que no tiene trabajo pero que busca trabajo y está disponible para realizarlo. Las definiciones de población activa y desempleo difieren según el país

https://datos.bancomundial.org/indicador/SL.UEM.TOTL.ZS?view=chart

In [246]:
url_24 ="https://api.worldbank.org/v2/es/indicator/SL.UEM.TOTL.ZS?downloadformat=csv"

In [247]:
response_url_24 = requests.get(url_24)

In [248]:
if response_url_24.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_24.content), "r") as zip_url_24:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_24.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_24.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [249]:
# Ruta del archivo CSV

csv_url_24 = "dataset/API_SL.UEM.TOTL.ZS_DS2_es_csv_v2_5456356.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_24 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_24, "r") as file_csv_url_24:
    csv_reader_url_24 = csv.reader(file_csv_url_24)
    rows_url_24 = list(csv_reader_url_24)

# Eliminar las primeras 4 filas

rows_url_24 = rows_url_24[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_24.csv", "w", newline="") as file_csv_url_24:
    csv_writer_url_24 = csv.writer(file_csv_url_24)
    csv_writer_url_24.writerows(rows_url_24)

In [250]:
data_24 = spark.read.csv("dataset/data/data_24.csv", header=True)

In [251]:
data_24 = data_24.drop("_c66","_c67")

In [252]:
data_24.limit(3).pandas_api()

Country Name Country Code                                                               Indicator Name  Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990              1991              1992              1993              1994              1995              1996              1997              1998              1999              2000              2001              2002              2003              2004              2005              2006              2007              2008              2009              2010              2011              2012              2013              2014              2015              2016              2017              2018              2019              2020              2021              2022
0        Aruba          ABW  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None              None
1         None          AFE  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  7.33333582862451  7.31874715974837  7.24270531831179  7.16069407881989  7.06379600929184  7.05599784776945  7.09054127946598  7.06009639229653  7.01527149574635  6.93953643462879  6.85037600360991  6.80353665561531  6.74124106047488  6.53517315091514  6.37350255682969  6.34759835935351  6.28342096946086  6.23256052686116  6.29558691357936  6.93364466215657  6.71535792804545  6.59935562174995  6.51278389070957  6.55564519787211  6.70714163907837  6.85558940196983  6.94036502294338  6.91304636428334  7.12166350612313  7.63130341080996  7.92021884481321  7.91683058027188
2   Afganistán          AFG  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None             8.121             8.168             8.123             8.111              8.26             8.165             8.089             8.082              8.07             8.054              8.04             8.186             8.122             8.053             8.113             8.054             8.108             8.022             8.082             8.068             7.947             8.019             7.949              7.91             8.989            10.086             11.18             11.11            11.085             11.71              None              None

In [253]:
data_24 = data_24.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_unemployment)")

In [254]:
data_24.sample(False, 3/data_24.count(), seed=150).limit(3).pandas_api()

Country Name Country Code                                                               Indicator Name  Indicator Code  Year porc_unemployment
0         None          AFE  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  1993  7.24270531831179
1   Cabo Verde          CPV  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  2016            12.002
2       Israel          ISR  Desempleo, total (% de la población activa total) (estimación modelado OIT)  SL.UEM.TOTL.ZS  1988              None

In [255]:
data_24 = data_24.drop("Country Name","Indicator Name","Indicator Code")
data_24.columns

['Country Code', 'Year', 'porc_unemployment']

# Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)


La tasa de pobreza nacional es el porcentaje de personas que vive debajo de la línea de pobreza nacional. Las estimaciones nacionales se basan en estimaciones de subgrupos ponderados según la población, obtenidas a partir de encuestas de los hogares.

https://datos.bancomundial.org/indicador/SI.POV.NAHC?end=2022&start=1985&view=chart

In [256]:
url_25 ="https://api.worldbank.org/v2/es/indicator/SI.POV.NAHC?downloadformat=csv"

In [257]:
response_url_25 = requests.get(url_25)

In [258]:
if response_url_25.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_25.content), "r") as zip_url_25:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_25.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_25.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [259]:
# Ruta del archivo CSV
csv_url_25 = "dataset/API_SI.POV.NAHC_DS2_es_csv_v2_5457611.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_25 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_25, "r") as file_csv_url_25:
    csv_reader_url_25 = csv.reader(file_csv_url_25)
    rows_url_25 = list(csv_reader_url_25)

# Eliminar las primeras 4 filas
rows_url_25 = rows_url_25[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_25.csv", "w", newline="") as file_csv_url_25:
    csv_writer_url_25 = csv.writer(file_csv_url_25)
    csv_writer_url_25.writerows(rows_url_25)

In [260]:
data_25 = spark.read.csv("dataset/data/data_25.csv", header=True)

In [261]:
data_25 = data_25.drop("_c66","_c67")

In [262]:
data_25.limit(3).pandas_api()

Country Name Country Code                                                                                       Indicator Name Indicator Code  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971  1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997  1998  1999  2000  2001  2002  2003  2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  2022
0        Aruba          ABW  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
1         None          AFE  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None
2   Afganistán          AFG  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  None  33.7  None  None  None  38.3  None  None  None  None  54.5  None  None  None  None  None  None

In [263]:
data_25 = data_25.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, porc_below_poverty_line)")

In [264]:
data_25.sample(False, 3/data_25.count(), seed=150).limit(3).pandas_api()
#pocos datos ###VER###

Country Name Country Code                                                                                       Indicator Name Indicator Code  Year porc_below_poverty_line
0         None          AFE  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  1993                    None
1   Cabo Verde          CPV  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  2016                    None
2       Israel          ISR  Tasa de incidencia de la pobreza, sobre la base de la línea de pobreza nacional (% de la población)    SI.POV.NAHC  1988                    None

In [265]:
data_25 = data_25.drop("Country Name","Indicator Name","Indicator Code")
data_25.columns

['Country Code', 'Year', 'porc_below_poverty_line']

# Crecimiento del PIB per cápita (% anual)

Tasa de crecimiento porcentual anual del PIB per cápita en moneda local, a precios constantes. El PIB per cápita es el producto interno bruto dividido por la población a mitad de año. El PIB a precio de comprador es la suma del valor agregado bruto de todos los productores residentes en la economía más todo impuesto a los productos, menos todo subsidio no incluido en el valor de los productos. Se calcula sin hacer deducciones por depreciación de bienes manufacturados o por agotamiento y degradación de recursos naturales.Los datos se expresan en dólares de los Estados Unidos a precios constantes del año 2010.

https://datos.bancomundial.org/indicador/NY.GDP.PCAP.KD.ZG?view=chart

In [266]:
url_26 ="https://api.worldbank.org/v2/es/indicator/NY.GDP.PCAP.KD.ZG?downloadformat=csv"

In [267]:
response_url_26 = requests.get(url_26)

In [268]:
if response_url_26.status_code == 200:
    # Crear un objeto ZipFile a partir de los datos de la respuesta
    with zipfile.ZipFile(io.BytesIO(response_url_26.content), "r") as zip_url_26:
        # Extraer todos los archivos del archivo ZIP en el directorio actual
        zip_url_26.extractall("dataset")

    print("El archivo ZIP se ha descargado y descomprimido correctamente.")
else:
    print("Error al descargar el archivo ZIP:", response_url_26.status_code)

El archivo ZIP se ha descargado y descomprimido correctamente.


In [269]:
# Ruta del archivo CSV
csv_url_26 = "dataset/API_NY.GDP.PCAP.KD.ZG_DS2_es_csv_v2_5478754.csv"

# Crear una lista para almacenar las filas del archivo CSV
rows_url_26 = []

# Leer el archivo CSV y almacenar las filas en la lista
with open(csv_url_26, "r") as file_csv_url_26:
    csv_reader_url_26 = csv.reader(file_csv_url_26)
    rows_url_26 = list(csv_reader_url_26)
    
# Eliminar las primeras 4 filas

rows_url_26 = rows_url_26[4:]

# Escribir las filas modificadas en un nuevo archivo CSV
with open("dataset/data/data_26.csv", "w", newline="") as file_csv_url_26:
    csv_writer_url_26 = csv.writer(file_csv_url_26)
    csv_writer_url_26.writerows(rows_url_26)

In [270]:
data_26 = spark.read.csv("dataset/data/data_26.csv", header=True)

In [271]:
data_26 = data_26.drop("_c66","_c67")

In [272]:
data_26.limit(3).pandas_api()

Country Name Country Code                            Indicator Name     Indicator Code  1960               1961              1962              1963              1964              1965              1966              1967              1968              1969              1970              1971                1972              1973              1974               1975                1976               1977               1978                1979              1980              1981               1982               1983                1984               1985                1986              1987              1988               1989               1990               1991               1992               1993                1994                1995               1996              1997                1998                1999               2000              2001              2002               2003               2004               2005                 2006              2007               2008               2009               2010               2011               2012              2013                2014                2015                2016                 2017                 2018                2019               2020              2021  2022
0        Aruba          ABW  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  None               None              None              None              None              None              None              None              None              None              None              None                None              None              None               None                None               None               None                None              None              None               None               None                None               None                None  16.2639407353318  18.8662781416001   11.6715725749289   2.19580450971051   4.53931728266622   2.37064150750518   4.09261476333727    4.80035199531828  -0.567201753935393  -1.83002373800004  3.85474445102984   -1.02708952603501   -1.68953665272255   4.91835385108675  2.36919299315137   -2.134006060887  0.117317036422548   6.31877676516297  -1.37914839490475  -0.0490843651328845  1.83873074125837  0.577087557491993  -12.7660428841211  -3.82701299563932   2.40034245209328  -1.83397735596419  5.63531750282364  -0.709567170830184    2.88516618544072    1.51593791406077     4.91713876742246     4.73833114017752   0.181215791943856  -18.6983248900374  17.2253003168657  None
1         None          AFE  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  None  -2.36009309307849  5.11213155041716  2.34603296692406  1.72825132841423  2.41855144484609  1.07018814600256  2.35805851973888  1.11265156396408  2.31279700654505  1.73175209827978  2.39047023895328  -0.696691317047112  1.49809420084335  2.81907458913334  -1.21028667292575  -0.114390862397272  -1.59471954451476  -1.91043997981032  -0.330075031335099  2.37936041329361  1.27369563721973  -2.53470140377524  -2.76734670272283  0.0730640742944786  -3.31517484647986  -0.706909874974571  1.14593673186863  1.06422626384158  0.063231054907618  -2.86037706819768  -2.60562090946952  -4.47952389209425  -2.96737099952324  -0.580264108057932      1.628756180678   2.66906474445234  1.80588905886579  -0.652243830755737  0.0903084878166425  0.750995727518642  1.03642073859993  1.26566153278462  0.466079711026453   2.78858920294005   3.36106865514503     3.75725646629159  3.71948950843135   1.49530847698216  -2.05372649707763   2.37374207253443  0.936366201114254  -1.91775331299382   1.4100493641305    1.22995929478684  0.0906019869672861  -0.235775006903822  -0.0157183331764656  -0.0755448262587493  -0.635728676015617  -5.57161969015738  1.74935984690359  None
2   Afganistán          AFG  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  None               None              None              None              None              None              None              None          

In [273]:
data_26 = data_26.selectExpr('`Country Name`', '`Country Code`', "`Indicator Name`", "`Indicator Code`", "stack(63, \
'1960', `1960`, '1961', `1961`, '1962', `1962`, '1963', `1963`, '1964', `1964`, \
'1965', `1965`, '1966', `1966`, '1967', `1967`, '1968', `1968`, '1969', `1969`, \
'1970', `1970`, '1971', `1971`, '1972', `1972`, '1973', `1973`, '1974', `1974`, \
'1975', `1975`, '1976', `1976`, '1977', `1977`, '1978', `1978`, '1979', `1979`, \
'1980', `1980`, '1981', `1981`, '1982', `1982`, '1983', `1983`, '1984', `1984`, \
'1985', `1985`, '1986', `1986`, '1987', `1987`, '1988', `1988`, '1989', `1989`, \
'1990', `1990`, '1991', `1991`, '1992', `1992`, '1993', `1993`, '1994', `1994`, \
'1995', `1995`, '1996', `1996`, '1997', `1997`, '1998', `1998`, '1999', `1999`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2010', `2010`, '2011', `2011`, '2012', `2012`, '2013', `2013`, '2014', `2014`, \
'2015', `2015`, '2016', `2016`, '2017', `2017`, '2018', `2018`, '2019', `2019`, \
'2020', `2020`, '2021', `2021`, '2022', `2022`) as (Year, PIB_growth_per_capita)")

In [274]:
data_26.sample(False, 3/data_26.count(), seed=150).limit(3).pandas_api()

Country Name Country Code                            Indicator Name     Indicator Code  Year PIB_growth_per_capita
0         None          AFE  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  1993     -2.96737099952324
1   Cabo Verde          CPV  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  2016      3.53796436657309
2       Israel          ISR  Crecimiento del PIB per cápita (% anual)  NY.GDP.PCAP.KD.ZG  1988                  None

In [275]:
data_26.count()

16758

In [276]:
data_26 = data_26.drop("Country Name","Indicator Name","Indicator Code")
data_26.columns

['Country Code', 'Year', 'PIB_growth_per_capita']

In [311]:
americans_countries = ['Argentina', 'Bahamas', 'Belice', 'Bolivia', 'Brasil', 'Canadá', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', \
                       'Dominica', 'Ecuador', 'El Salvador', 'Estados Unidos', 'Granada', 'Guatemala', 'Guyana', 'Haití', 'Honduras', \
                       'Jamaica', 'México', 'Nicaragua', 'Panamá', 'Paraguay', 'Perú', 'República Dominicana', 'San Vicente y las Granadinas', \
                       'Santa Lucía', 'Suriname', 'Trinidad y Tobago', 'Uruguay', 'Venezuela']

oceanic_countries = ['Australia', 'Fiyi', 'Islas Marshall', 'Islas Salomón', 'Kiribati', 'Micronesia (Estados Federados de)', 'Nauru', \
                     'Nueva Zelanda', 'Palaos', 'Papúa Nueva Guinea', 'Samoa', 'Tonga', 'Tuvalu', 'Vanuatu']

african_contries = ["Argelia", "Angola", "Benín", "Botsuana", "Burkina Faso", "Burundi", "Cabo Verde", "Camerún", "Chad", "Comoras", "República Democrática del Congo", \
                    "Yibuti", "Egipto", "Guinea Ecuatorial", "Eritrea", "Eswatini", "Etiopía", "Gabón", "Gambia", "Ghana", "Guinea", "Guinea-Bisáu", "Costa de Marfil", \
                    "Kenia", "Lesoto", "Liberia", "Libia", "Madagascar", "Malaui", "Malí", "Mauritania", "Mauricio", "Marruecos", "Mozambique", "Namibia", "Níger", \
                    "Nigeria", "República del Congo", "Ruanda", "Santo Tomé y Príncipe", "Senegal", "Seychelles", "Sierra Leona", "Somalia", "Sudáfrica", "Sudán", \
                    "Sudán del Sur", "Tanzania", "Togo", "Túnez", "Uganda", "Zambia", "Zimbabue"]

african_countries_who = [ "Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cape Verde", "Cameroon", "Chad", "Comoros", "Democratic Republic of the Congo",
                         "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast",
                         "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger",
                         "Nigeria", "Republic of the Congo", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa",
                         "South Sudan", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe" ]

In [278]:
data = data_00.join(data_01, ["Year", "Country Code"], "fullouter") \
    .join(data_02, ["Year", "Country Code"], "fullouter") \
    .join(data_03, ["Year", "Country Code"], "fullouter") \
    .join(data_04, ["Year", "Country Code"], "fullouter") \
    .join(data_05, ["Year", "Country Code"], "fullouter") \
    .join(data_06, ["Year", "Country Code"], "fullouter") \
    .join(data_07, ["Year", "Country Code"], "fullouter") \
    .join(data_08, ["Year", "Country Code"], "fullouter") \
    .join(data_09, ["Year", "Country Code"], "fullouter") \
    .join(data_10, ["Year", "Country Code"], "fullouter") 

In [279]:
#data = data.filter(col("Country Name").isin(americans_countries) | col("Country Name").isin(oceanic_countries))
data = data.filter(col("Year") >= "1985")
data = data.drop_duplicates()   

In [280]:
data = data.join(data_11, ["Year", "Country Code"], "fullouter") \
    .join(data_12, ["Year", "Country Code"], "fullouter") \
    .join(data_13, ["Year", "Country Code"], "fullouter") \
    .join(data_14, ["Year", "Country Code"], "fullouter") \
    .join(data_15, ["Year", "Country Code"], "fullouter") \
    .join(data_16, ["Year", "Country Code"], "fullouter") \
    .join(data_17, ["Year", "Country Code"], "fullouter") \
    .join(data_18, ["Year", "Country Code"], "fullouter") \
    .join(data_19, ["Year", "Country Code"], "fullouter") \
    .join(data_20, ["Year", "Country Code"], "fullouter") 

In [281]:
#data = data.filter(col("Country Name").isin(americans_countries) | col("Country Name").isin(oceanic_countries))
data = data.filter(col("Year") >= "1985")
data = data.drop_duplicates()   

In [282]:
data = data.join(data_22, ["Year", "Country Code"], "fullouter") \
    .join(data_23, ["Year", "Country Code"], "fullouter") \
    .join(data_24, ["Year", "Country Code"], "fullouter") \
    .join(data_25, ["Year", "Country Code"], "fullouter") \
    .join(data_26, ["Year", "Country Code"], "fullouter")

In [283]:
#data = data.filter(col("Country Name").isin(americans_countries) | col("Country Name").isin(oceanic_countries))
data = data.filter(col("Year") >= "1985")
data = data.drop_duplicates()   

In [284]:
#data.coalesce(1).write.option("header", "true").mode("overwrite").parquet("data")

In [285]:
# Contar Numero de filas

data.count()

10108

In [286]:
# Listar columnas

print(data.columns)

['Year', 'Country Code', 'Country Name', 'life_expatancy', 'life_expatancy_male', 'life_expatancy_female', 'ratio_mort_neonatal', 'ratio_mort_one_year', 'ratio_mort_five_year', 'ratio_maternal_mort', 'ratio_mort_in_year', 'ratio_poblation_0_14_year', 'ratio_poblation_15_64_year', 'ratio_poblation_more_64_year', 'ratio_population_growth', 'ratio_mortality_CVD_cancer_diabetes_CRD', 'ratio_infectious_diseases', 'porc_prev_malnutrition', 'doctor_1000_people', 'bed_1000_people', 'porc_water_service', 'porc_sanitation_services', 'porc_electricity', 'porc_internet', 'porc_secundary_school_compl', 'porc_literacy', 'porc_unemployment', 'porc_below_poverty_line', 'PIB_growth_per_capita']


In [287]:
data.printSchema

<bound method DataFrame.printSchema of DataFrame[Year: string, Country Code: string, Country Name: string, life_expatancy: string, life_expatancy_male: string, life_expatancy_female: string, ratio_mort_neonatal: string, ratio_mort_one_year: string, ratio_mort_five_year: string, ratio_maternal_mort: string, ratio_mort_in_year: string, ratio_poblation_0_14_year: string, ratio_poblation_15_64_year: string, ratio_poblation_more_64_year: string, ratio_population_growth: string, ratio_mortality_CVD_cancer_diabetes_CRD: string, ratio_infectious_diseases: string, porc_prev_malnutrition: string, doctor_1000_people: string, bed_1000_people: string, porc_water_service: string, porc_sanitation_services: string, porc_electricity: string, porc_internet: string, porc_secundary_school_compl: string, porc_literacy: string, porc_unemployment: string, porc_below_poverty_line: string, PIB_growth_per_capita: string]>

In [288]:
data.sample(False, 3/data.count(), seed=150).limit(3).pandas_api()

Year Country Code Country Name    life_expatancy life_expatancy_male life_expatancy_female ratio_mort_neonatal ratio_mort_one_year ratio_mort_five_year ratio_maternal_mort ratio_mort_in_year ratio_poblation_0_14_year ratio_poblation_15_64_year ratio_poblation_more_64_year ratio_population_growth ratio_mortality_CVD_cancer_diabetes_CRD ratio_infectious_diseases porc_prev_malnutrition doctor_1000_people bed_1000_people porc_water_service porc_sanitation_services  porc_electricity porc_internet porc_secundary_school_compl porc_literacy porc_unemployment porc_below_poverty_line PIB_growth_per_capita
0  2008          POL      Polonia  75.5439024390244                None                  None                None                None                 None                None               None                      None                       None                         None                    None                                    None                      None                   None               None            None               None                     None              None          None                        None          None              None                    None                  None
1  1987          BDI      Burundi            47.516               45.73                49.282                39.6                39.6                168.9                None             17.569          47.2987397553806           49.6574536061477             3.04380663847166        2.27596047847185                                    None                      None                   None               None            None               None                     None              None             0            2.24618005752563          None              None                    None      3.12900649958789
2  2020          KHM      Camboya            70.416              67.999                72.724                13.3                13.3                 25.8                 218              6.228          29.6255402063096           65.1018681289629             5.27259166472753        1.16005769739725                                    None                      None                    6.3               None            None               None              68.76930002  86.4000015258789   53.65076745             58.158390045166          None             0.298                    None     -4.21365375468523

In [312]:
import dropbox
from dropbox.files import WriteMode
from io import BytesIO

app_key = "eefsz35wynsn1yo"
app_secret = "pjytroccqrvqpns"

auth_flow = dropbox.DropboxOAuth2FlowNoRedirect(app_key, app_secret)
auth_url = auth_flow.start()

print("Ir a la siguiente URL y haz clic en `Permitir`:")
print(auth_url)

auth_code = input("Introduce el código de autorización aquí: ").strip()

Ir a la siguiente URL y haz clic en `Permitir`:
https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=eefsz35wynsn1yo


Introduce el código de autorización aquí:  W0Lh7X4IHXYAAAAAAAACqrjjOOYTYfCQ4XlAVS_nTSg


In [307]:
auth_result = auth_flow.finish(auth_code)
access_token = auth_result.access_token

# Crea una instancia de la API de Dropbox utilizando el token de acceso
dbx = dropbox.Dropbox(access_token)

# Escribir el DataFrame en una ubicación temporal
temp_csv_path = "/_disk_dev/final_project/temp.csv"
data.coalesce(1).write.option("header", "true").mode("overwrite").csv(temp_csv_path)

El token de acceso es:
sl.BgFLH3rfPXgDVQCCJndYF9o73Mgx2nweHnGeEMIYmc0D9hk6igcIGCuSMY7Vm9029x7gja6GcwjnbJGYJoUyWMPjBrS1R4keqIxAZ9Pf0XmHCNWe0VFkm_Pm1pahfC3LP68b-b4


23/06/11 08:10:46 WARN DAGScheduler: Broadcasting large task binary with size 1018.3 KiB


In [308]:
import os
from dropbox import Dropbox
from io import BytesIO

# Obtener el nombre del archivo CSV generado por Spark
csv_files = [file for file in os.listdir(temp_csv_path) if file.endswith(".csv")]
if len(csv_files) == 0:
    raise FileNotFoundError("No se encontró el archivo CSV generado por Spark")
csv_file = os.path.join(temp_csv_path, csv_files[0])

In [309]:
# Leer el archivo CSV en memoria
with open(csv_file, "rb") as file:
    csv_content = file.read()
    
# Subir el archivo CSV a Dropbox
# Ruta de la carpeta en Dropbox donde deseas guardar el archivo
folder_path = "/life_expectancy"  
# Nombre del archivo
file_name = "data1.csv"  # Nombre del archivo
file_path = folder_path + "/" + file_name

In [310]:
dbx.files_upload(csv_content, file_path, mode=WriteMode.overwrite)

FileMetadata(client_modified=datetime.datetime(2023, 6, 11, 11, 9, 5), content_hash='8b337efe75fdffe7c676bc97120395a8c76b1495711c5cba0e5f94966649947d', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:jSc0qDJVbngAAAAAAAB2pw', is_downloadable=True, media_info=NOT_SET, name='data1.csv', parent_shared_folder_id=NOT_SET, path_display='/life_expectancy/data1.csv', path_lower='/life_expectancy/data1.csv', preview_url=NOT_SET, property_groups=NOT_SET, rev='5fdd8a15f20851359cdd7', server_modified=datetime.datetime(2023, 6, 11, 11, 9, 5), sharing_info=NOT_SET, size=1741142, symlink_info=NOT_SET)

In [294]:
import dropbox
import tempfile

In [295]:
ruta_archivo_dropbox = '/life_expectancy/hola_drop.py'  # Reemplaza con la ruta correcta del archivo en Dropbox

# Crear el archivo temporal
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    # Descargar el archivo de Dropbox
    metadata, res = dbx.files_download(ruta_archivo_dropbox)

    # Escribir los datos descargados en el archivo temporal
    temp_file.write(res.content)

    # Obtener la ruta del archivo temporal
    temp_file_path = temp_file.name

In [296]:
print(temp_file_path)

/tmp/tmp2n128kgw


In [297]:
# Ejecutar el contenido del archivo temporal
with open(temp_file_path, 'r') as temp_file:
    codigo = temp_file.read()
    exec(codigo)

Grande


In [298]:
import dropbox
import subprocess


In [299]:
# Ruta y nombre del archivo en Dropbox
ruta_archivo = "/life_expectancy/hola_drop.py"

# Obtener el enlace de descarga directa del archivo
enlace_descarga = dbx.files_get_temporary_link(ruta_archivo).link

# Ejecutar el archivo Python utilizando el enlace de descarga directa
subprocess.call(["python", enlace_descarga])

python: can't open file '/_disk_dev/final_project/https://uc358c27be3a524df43bd5909ceb.dl.dropboxusercontent.com/cd/0/get/B9x3NLvIV-p0NAjc2Qw6HGeNgpOeCT8dxlAgBkDui5iw4M52JaA4gyLg1gB7qSK6CMkEQRr3-jPUYkuSEnSY0OmFvsPxFUtE-0lmqkYD5ZtzpYQlp97qK0xzegtNT8lUhC7hFGTw_W_whpth2ELqkwKLnBl81QLXD2qhs8N-dvq_7Ra1eoQrFXjilqK0x59mtKY/file': [Errno 2] No such file or directory


2

In [300]:
import dropbox
import subprocess
import requests

# Initialize Dropbox API client
#dbx = dropbox.Dropbox("YOUR_DROPBOX_ACCESS_TOKEN")

# Ruta y nombre del archivo en Dropbox
ruta_archivo = "/life_expectancy/hola_drop.py"

# Obtener el enlace de descarga directa del archivo
metadata, response = dbx.files_download(ruta_archivo)
contenido_archivo = response.content

# Guardar el contenido del archivo en un archivo local
ruta_local_archivo = "hola_drop.py"
with open(ruta_local_archivo, "wb") as file:
    file.write(contenido_archivo)

# Ejecutar el archivo Python localmente
subprocess.call(["python", ruta_local_archivo])


Grande


0

In [301]:
import urllib.request
import subprocess

In [302]:
# Ruta y nombre del archivo en Dropbox
ruta_archivo = "/life_expectancy/hola_drop.py"

# Obtener el enlace de descarga directa del archivo
enlace_descarga = dbx.files_get_temporary_link(ruta_archivo).link

In [303]:
print(enlace_descarga)

https://ucdc92095997f341e98b195201cd.dl.dropboxusercontent.com/cd/0/get/B9zUfT39XGEsSQ13LQ_xCM6Vee3Y_3aIm5kin8eqS0JzmNKXa2XD1fa6axFmTvp5iJnkHQ-sYDFzfYyjiDW5AA6D4jppLSI15PMZtuBi5DD_1O53W9aBcfyuo-iO0zHISGlB3HBo92-ciN63AcjYHweijZSlYKMC-vzd3vkpcy5sQYO0m4j7RnedFemo2yqGMpw/file
